# Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import os
import math as mt
from astropy.modeling import models
from astropy import units as u
from scipy.optimize import curve_fit
from scipy.optimize import newton, least_squares
from astropy.timeseries import LombScargle
from scipy.special import voigt_profile
import Analysis_functions as AF
from importlib import reload
import scipy.stats as ss
import matplotlib.colors as mc
import csv
import pandas as pd
from scipy.interpolate import interp1d
import bindensity as bind
import itertools
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
import spectrum_model as spec_mod
import line_model as line_mod

# Get the spectra for a given day

In [ ]:
day = '2023-01-26'
new = True
main_directory = '/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/new_Solar-data'
location = main_directory+'/'+day
os.chdir(location)
#if not os.path.exists('Plot_Results'):
#    os.mkdir('Plot_Results')
    
#if not os.path.exists('Final_Plot_Results'):
#    os.mkdir('Final_Plot_Results')

if not os.path.exists('Big_Exclusion_Final_Plot_Results'):
    os.mkdir('Big_Exclusion_Final_Plot_Results')

save_location = location + '/Big_Exclusion_Final_Plot_Results/'

#Defining directories
blaze_dir = 'Blaze-data'
telluric_dir = 'Telluric-data'
dir_name = 'Corr-data'
CCF_dir_name = 'CCF_RV-data'
rassine_dir_name = 'Big-Exclusion-Rassine-data'
Rassine = True

#Continuum Ranges used for normalization and EW calculation
normalization_poly_order = 0
fit_poly_order = 0
wav_ranges = [(10804.8, 10805.9), (10807.4, 10808.5), (10825.5, 10826.5), (10839, 10840), (10842.4, 10843.1), (10850.2, 10851.2), (10856.5, 10857.9)]

# Extract data

In [ ]:
reload(AF)
save = True

#Extracting the data
if new:
    print('New Extraction')
    if Rassine:
        print('Rassine')
        if os.path.exists(location+'/'+rassine_dir_name+'-HA'):
            print('Multiple modes')
            tot_lamda, rassine_total_lamda_HA, rassine_total_lamda_HE, rassine_master_lamda_HA, rassine_master_lamda_HE, tot_spctr, tot_norm_spctr, rassine_total_norm_spctr_HA, rassine_total_norm_spctr_HE, rassine_master_spctr_HA, rassine_master_spctr_HE, rassine_continuum_HA, rassine_continuum_HE, tot_err, tot_norm_err, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM, tot_telluric = AF.new_extraction(location, dir_name, blaze_dir, CCF_dir_name, telluric_dir, rassine_dir_name, 14, wav_ranges, normalization_poly_order, save_location, Rassine, save)
        else:
            print('Single mode')
            tot_lamda, rassine_total_lamda, rassine_master_lamda, tot_spctr, tot_norm_spctr, rassine_total_norm_spctr, rassine_master_spctr, rassine_continuum, tot_err, tot_norm_err, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM, tot_telluric = AF.new_extraction(location, dir_name, blaze_dir, CCF_dir_name, telluric_dir, rassine_dir_name, 14, wav_ranges, normalization_poly_order, save_location, Rassine, save)
    else:
        print('No Rassine')
        tot_lamda, tot_spctr, tot_norm_spctr, tot_err, tot_norm_err, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM, tot_telluric = AF.new_extraction(location, dir_name, blaze_dir, CCF_dir_name, telluric_dir, rassine_dir_name, 14, wav_ranges, normalization_poly_order, save_location, Rassine, save)
else:
    print('Old Extraction')
    tot_lamda, tot_norm_spctr, tot_norm_err, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM = AF.extraction(dir_name, blaze_dir, CCF_dir_name, 14)


# Data reduction

## SNR and RV clipping

In [ ]:
reload(AF)
sig = 200
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(tot_mode=='A') != len(tot_mode) and np.sum(tot_mode=='E')!= len(tot_mode):
    total_SNR_HA, total_SNR_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_SNR, tot_RV, tot_AIRM, sig)
    
    #Special ones - BEGIN
    tota_norm_spctr_HA, tota_norm_spctr_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_spctr, tot_RV, tot_AIRM, sig)

    tota_lamda_HA, tota_lamda_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_lamda, tot_RV, tot_AIRM, sig)
    
    tota_norm_err_HA, tota_norm_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_err, tot_RV, tot_AIRM, sig)
    
    tota_spctr_HA, tota_spctr_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_spctr, tot_RV, tot_AIRM, sig)
        
    tota_err_HA, tota_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_err, tot_RV, tot_AIRM, sig)
    
    tota_telluric_HA, tota_telluric_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_telluric, tot_RV, tot_AIRM, sig)
    #Special ones - END

    date_HA, date_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_date, tot_RV, tot_AIRM, sig)

    total_RV_HA, total_RV_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV, tot_RV, tot_AIRM, sig)

    total_RV_err_HA, total_RV_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV_err, tot_RV, tot_AIRM, sig)

    total_FWHM_HA, total_FWHM_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM, tot_RV, tot_AIRM, sig)

    total_FWHM_err_HA, total_FWHM_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM_err, tot_RV, tot_AIRM, sig)

    total_BIS_SPAN_HA, total_BIS_SPAN_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN, tot_RV, tot_AIRM, sig)

    total_BIS_SPAN_err_HA, total_BIS_SPAN_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN_err, tot_RV, tot_AIRM, sig)

    total_CONTRAST_HA, total_CONTRAST_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST, tot_RV, tot_AIRM, sig)

    total_CONTRAST_err_HA, total_CONTRAST_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST_err, tot_RV, tot_AIRM, sig)

    total_H2O_HA, total_H2O_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O, tot_RV, tot_AIRM, sig)

    total_H2O_err_HA, total_H2O_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O_err, tot_RV, tot_AIRM, sig)

    total_O2_HA, total_O2_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2, tot_RV, tot_AIRM, sig)
    
    total_O2_err_HA, total_O2_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2_err, tot_RV, tot_AIRM, sig)

    total_CO2_HA, total_CO2_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2, tot_RV, tot_AIRM, sig)

    total_CO2_err_HA, total_CO2_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2_err, tot_RV, tot_AIRM, sig)

    total_AIRM_HA, total_AIRM_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_AIRM, tot_RV, tot_AIRM, sig)
    
#If there is one mode of observation
else:
    #We don't need to segment and reduce the RASSINE spectra because they were made from 
    #spectra that were previously segmented and reduced. 
    
    #Special ones - BEGIN
    tota_norm_spctr = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_spctr, tot_RV, tot_AIRM, sig)

    tota_lamda = AF.segment_and_reduce(tot_mode, tot_SNR, tot_lamda, tot_RV, tot_AIRM, sig)
    
    tota_norm_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_err, tot_RV, tot_AIRM, sig)
    
    tota_spctr = AF.segment_and_reduce(tot_mode, tot_SNR, tot_spctr, tot_RV, tot_AIRM, sig)
        
    tota_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_err, tot_RV, tot_AIRM, sig)
    
    tota_telluric = AF.segment_and_reduce(tot_mode, tot_SNR, tot_telluric, tot_RV, tot_AIRM, sig)
    #Special ones - END
    
    total_SNR = AF.segment_and_reduce(tot_mode, tot_SNR, tot_SNR, tot_RV, tot_AIRM, sig)

    date = AF.segment_and_reduce(tot_mode, tot_SNR, tot_date, tot_RV, tot_AIRM, sig)

    total_RV = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV, tot_RV, tot_AIRM, sig)

    total_RV_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV_err, tot_RV, tot_AIRM, sig)

    total_FWHM = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM, tot_RV, tot_AIRM, sig)

    total_FWHM_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM_err, tot_RV, tot_AIRM, sig)

    total_BIS_SPAN = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN, tot_RV, tot_AIRM, sig)

    total_BIS_SPAN_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN_err, tot_RV, tot_AIRM, sig)

    total_CONTRAST = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST, tot_RV, tot_AIRM, sig)

    total_CONTRAST_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST_err, tot_RV, tot_AIRM, sig)

    total_H2O = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O, tot_RV, tot_AIRM, sig)

    total_H2O_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O_err, tot_RV, tot_AIRM, sig)

    total_O2 = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2, tot_RV, tot_AIRM, sig)
    
    total_O2_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2_err, tot_RV, tot_AIRM, sig)

    total_CO2 = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2, tot_RV, tot_AIRM, sig)

    total_CO2_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2_err, tot_RV, tot_AIRM, sig)

    total_AIRM = AF.segment_and_reduce(tot_mode, tot_SNR, tot_AIRM, tot_RV, tot_AIRM, sig)
    
mode = tot_mode

## Making the Rassine errorbars

In [ ]:
if np.sum(tot_mode=='A') != len(tot_mode) and np.sum(tot_mode=='E')!= len(tot_mode):
    #HA
    #Define the wavelength solution onto which we want to interpolate the error bars
    rassine_wav_sol_HA = np.append(rassine_total_lamda_HA[0], rassine_total_lamda_HA[0][-1] + np.diff(rassine_total_lamda_HA[0])[-1])

    #Initialize the Rassine error bars
    rassine_tota_norm_err_HA = np.zeros(rassine_total_norm_spctr_HA.shape)

    for i in range(rassine_total_norm_spctr_HA.shape[0]):
        #Define the edges of the wavelength solution.
        base_wav_sol_HA = np.append(tota_lamda_HA[i], tota_lamda_HA[i][-1] + np.diff(tota_lamda_HA[i])[-1])
        rassine_tota_norm_err_HA[i] = bind.resampling(rassine_wav_sol_HA, base_wav_sol_HA, tota_err_HA[i], kind='cubic')/rassine_continuum_HA[i]       
        
    #Removing the nan values
    rassine_total_norm_err_HA = np.zeros((rassine_total_norm_spctr_HA.shape[0], rassine_total_norm_spctr_HA.shape[1]-2))
    new_rassine_total_lamda_HA = np.zeros((rassine_total_norm_spctr_HA.shape[0], rassine_total_norm_spctr_HA.shape[1]-2))
    new_rassine_total_norm_spctr_HA = np.zeros((rassine_total_norm_spctr_HA.shape[0], rassine_total_norm_spctr_HA.shape[1]-2))
    for i in range(rassine_total_norm_spctr_HA.shape[0]):
        rassine_total_norm_err_HA[i] = rassine_tota_norm_err_HA[i][1:-1]
        new_rassine_total_lamda_HA[i] = rassine_total_lamda_HA[i][1:-1]
        new_rassine_total_norm_spctr_HA[i] = rassine_total_norm_spctr_HA[i][1:-1]
    
    rassine_master_spctr_HA = rassine_master_spctr_HA[1:-1]
    rassine_master_lamda_HA = rassine_master_lamda_HA[1:-1]
    rassine_total_lamda_HA = new_rassine_total_lamda_HA
    rassine_total_norm_spctr_HA = new_rassine_total_norm_spctr_HA
    
    
    #HE
    #Define the wavelength solution onto which we want to interpolate the error bars
    rassine_wav_sol_HE = np.append(rassine_total_lamda_HE[0], rassine_total_lamda_HE[0][-1] + np.diff(rassine_total_lamda_HE[0])[-1])

    #Initialize the Rassine error bars
    rassine_tota_norm_err_HE = np.zeros(rassine_total_norm_spctr_HE.shape)

    for i in range(rassine_total_norm_spctr_HE.shape[0]):
        #Define the edges of the wavelength solution.
        base_wav_sol_HE = np.append(tota_lamda_HE[i], tota_lamda_HE[i][-1] + np.diff(tota_lamda_HE[i])[-1])
        rassine_tota_norm_err_HE[i] = bind.resampling(rassine_wav_sol_HE, base_wav_sol_HE, tota_err_HE[i], kind='cubic')/rassine_continuum_HE[i]

    #Removing the nan values
    rassine_total_norm_err_HE = np.zeros((rassine_total_norm_spctr_HE.shape[0], rassine_total_norm_spctr_HE.shape[1]-2))
    new_rassine_total_lamda_HE = np.zeros((rassine_total_norm_spctr_HE.shape[0], rassine_total_norm_spctr_HE.shape[1]-2))
    new_rassine_total_norm_spctr_HE = np.zeros((rassine_total_norm_spctr_HE.shape[0], rassine_total_norm_spctr_HE.shape[1]-2))
    for i in range(rassine_total_norm_spctr_HE.shape[0]):
        rassine_total_norm_err_HE[i] = rassine_tota_norm_err_HE[i][1:-1]
        new_rassine_total_lamda_HE[i] = rassine_total_lamda_HE[i][1:-1]
        new_rassine_total_norm_spctr_HE[i] = rassine_total_norm_spctr_HE[i][1:-1]
    
    rassine_master_spctr_HE = rassine_master_spctr_HE[1:-1]
    rassine_master_lamda_HE = rassine_master_lamda_HE[1:-1]
    rassine_total_lamda_HE = new_rassine_total_lamda_HE
    rassine_total_norm_spctr_HE = new_rassine_total_norm_spctr_HE

else:
    #Define the wavelength solution onto which we want to interpolate the error bars
    rassine_wav_sol = np.append(rassine_total_lamda[0], rassine_total_lamda[0][-1] + np.diff(rassine_total_lamda[0])[-1])

    #Initialize the Rassine error bars
    rassine_tota_norm_err = np.zeros(rassine_total_norm_spctr.shape)

    for i in range(rassine_total_norm_spctr.shape[0]):
        #Define the edges of the wavelength solution.
        base_wav_sol = np.append(tota_lamda[i], tota_lamda[i][-1] + np.diff(tota_lamda[i])[-1])
        rassine_tota_norm_err[i] = bind.resampling(rassine_wav_sol, base_wav_sol, tota_err[i], kind='cubic')/rassine_continuum[i]
        
    #Removing the nan values
    rassine_total_norm_err = np.zeros((rassine_total_norm_spctr.shape[0], rassine_total_norm_spctr.shape[1]-2))
    new_rassine_total_lamda = np.zeros((rassine_total_norm_spctr.shape[0], rassine_total_norm_spctr.shape[1]-2))
    new_rassine_total_norm_spctr = np.zeros((rassine_total_norm_spctr.shape[0], rassine_total_norm_spctr.shape[1]-2))
    for i in range(rassine_total_norm_spctr.shape[0]):
        rassine_total_norm_err[i] = rassine_tota_norm_err[i][1:-1]
        new_rassine_total_lamda[i] = rassine_total_lamda[i][1:-1]
        new_rassine_total_norm_spctr[i] = rassine_total_norm_spctr[i][1:-1]
    
    rassine_master_spctr = rassine_master_spctr[1:-1]
    rassine_master_lamda = rassine_master_lamda[1:-1]
    rassine_total_lamda = new_rassine_total_lamda
    rassine_total_norm_spctr = new_rassine_total_norm_spctr

## Interpolating everything on a common wavelength grid

In [ ]:
reload(AF)

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(tot_mode=='A') != len(tot_mode) and np.sum(tot_mode=='E')!= len(tot_mode):

    min_lamda = max(max(tota_lamda_HA[:, 0]), max(tota_lamda_HE[:, 0]))
    max_lamda = min(min(tota_lamda_HA[:, -1]), min(tota_lamda_HE[:, -1]))
    
    ref_wave = np.linspace(min_lamda, max_lamda, len(tota_lamda_HA[0]))
    
    total_lamda_HA = np.zeros(tota_lamda_HA.shape)
    total_lamda_HE = np.zeros(tota_lamda_HE.shape)
    
    total_norm_spctr_HA = np.zeros(tota_norm_spctr_HA.shape)
    total_norm_spctr_HE = np.zeros(tota_norm_spctr_HE.shape)
    
    total_norm_err_HA = np.zeros(tota_norm_err_HA.shape)
    total_norm_err_HE = np.zeros(tota_norm_err_HE.shape)
    
    total_spctr_HA = np.zeros(tota_spctr_HA.shape)
    total_spctr_HE = np.zeros(tota_spctr_HE.shape)

    total_err_HA = np.zeros(tota_err_HA.shape)
    total_err_HE = np.zeros(tota_err_HE.shape)

    #ANTARESS
    #Re-making the wavelength array so that it is the correct format for Antaress
    preliminary_lamda_HA=np.zeros((tota_lamda_HA.shape[0], tota_lamda_HA.shape[1]+1))

    preliminary_lamda_HE=np.zeros((tota_lamda_HE.shape[0], tota_lamda_HE.shape[1]+1))

    for i in range(len(preliminary_lamda_HA)):
        preliminary_lamda_HA[i] = np.append(tota_lamda_HA[i], tota_lamda_HA[i][-1]+np.diff(tota_lamda_HA[i])[-1])

    for i in range(len(preliminary_lamda_HE)):
        preliminary_lamda_HE[i] = np.append(tota_lamda_HE[i], tota_lamda_HE[i][-1]+np.diff(tota_lamda_HE[i])[-1])

    #Re-making the reference wavelength vector for Antaress
    ref_wave_Antaress = np.append(ref_wave, ref_wave[-1]+np.diff(ref_wave)[-1])

    #Making the bins for Rassine (telluric model on rassine wavelength solution)
    #Since all wavelength solutions of Rassine are the same we can set it to any of them
    ref_wave_Rassine_HA = np.append(rassine_total_lamda_HA[0], rassine_total_lamda_HA[0][-1] + np.diff(rassine_total_lamda_HA[0])[-1])

    ref_wave_Rassine_HE = np.append(rassine_total_lamda_HE[0], rassine_total_lamda_HE[0][-1] + np.diff(rassine_total_lamda_HE[0])[-1])

    #Initializing variables
    #Normalized
    total_norm_spctr_Antaress_HA = np.zeros(tota_norm_spctr_HA.shape)
    total_norm_spctr_Antaress_HE = np.zeros(tota_norm_spctr_HE.shape)

    total_norm_err_Antaress_HA = np.zeros(tota_norm_err_HA.shape)
    total_norm_err_Antaress_HE = np.zeros(tota_norm_err_HE.shape)

    #Un-normalized (for SNR calculation)
    total_spctr_Antaress_HA = np.zeros(tota_norm_spctr_HA.shape)
    total_spctr_Antaress_HE = np.zeros(tota_norm_spctr_HE.shape)

    total_err_Antaress_HA = np.zeros(tota_norm_err_HA.shape)
    total_err_Antaress_HE = np.zeros(tota_norm_err_HE.shape)
    
    total_telluric_HA = np.zeros(tota_telluric_HA.shape)
    total_telluric_HE = np.zeros(tota_telluric_HE.shape)

    rassine_total_telluric_HA = np.zeros(rassine_total_lamda_HA.shape)
    rassine_total_telluric_HE = np.zeros(rassine_total_lamda_HE.shape)
   
    #HA
    for i in range(len(tota_norm_spctr_HA)):
        
        interpol_s = interp1d(tota_lamda_HA[i], tota_norm_spctr_HA[i])
        total_norm_spctr_HA[i] = interpol_s(ref_wave)
        
        interpol_err = interp1d(tota_lamda_HA[i], tota_norm_err_HA[i]**2)
        total_norm_err_HA[i] = np.sqrt(interpol_err(ref_wave))
            
        interpol_spctr = interp1d(tota_lamda_HA[i], tota_spctr_HA[i])
        total_spctr_HA[i] = interpol_spctr(ref_wave)

        interpol_error = interp1d(tota_lamda_HA[i], tota_err_HA[i]**2)
        total_err_HA[i] = np.sqrt(interpol_error(ref_wave))

        total_lamda_HA[i] = ref_wave
        
        #Populating ANTARESS arrays
        #Normalized
        testA = bind.resampling(ref_wave_Antaress, preliminary_lamda_HA[i], tota_norm_spctr_HA[i], cov = np.array([tota_norm_err_HA[i]**2]) , kind='cubic')

        total_norm_spctr_Antaress_HA[i] = testA[0]

        total_norm_err_Antaress_HA[i] = np.sqrt(testA[1][0])
        
        #Un-normalized
        testB = bind.resampling(ref_wave_Antaress, preliminary_lamda_HA[i], tota_spctr_HA[i], cov = np.array([tota_err_HA[i]**2]) , kind='cubic')
        
        total_spctr_Antaress_HA[i] = testB[0]
        
        total_err_Antaress_HA[i] = np.sqrt(testB[1][0])
        
        testC = bind.resampling(ref_wave_Antaress, preliminary_lamda_HA[i], tota_telluric_HA[i], kind='cubic')
        
        total_telluric_HA[i] = testC
        
        testZ = bind.resampling(ref_wave_Rassine_HA, preliminary_lamda_HA[i], tota_telluric_HA[i], kind='cubic')
        
        rassine_total_telluric_HA[i] = testZ
       
    #HE
    for i in range(len(tota_norm_spctr_HE)):
        
        interpol_s = interp1d(tota_lamda_HE[i], tota_norm_spctr_HE[i])
        total_norm_spctr_HE[i] = interpol_s(ref_wave)
        
        interpol_err = interp1d(tota_lamda_HE[i], tota_norm_err_HE[i]**2)
        total_norm_err_HE[i] = np.sqrt(interpol_err(ref_wave))
        
        interpol_spctr = interp1d(tota_lamda_HE[i], tota_spctr_HE[i])
        total_spctr_HE[i] = interpol_spctr(ref_wave)

        interpol_error = interp1d(tota_lamda_HE[i], tota_err_HE[i]**2)
        total_err_HE[i] = np.sqrt(interpol_error(ref_wave))
        
        total_lamda_HE[i] = ref_wave
        
        #Populating ANTARESS arrays
        #Normalized
        testE = bind.resampling(ref_wave_Antaress, preliminary_lamda_HE[i], tota_norm_spctr_HE[i], cov = np.array([tota_norm_err_HE[i]**2]) , kind='cubic')

        total_norm_spctr_Antaress_HE[i] = testE[0]

        total_norm_err_Antaress_HE[i] = np.sqrt(testE[1][0])
        
        #Un-normalized
        testF = bind.resampling(ref_wave_Antaress, preliminary_lamda_HE[i], tota_spctr_HE[i], cov = np.array([tota_err_HE[i]**2]) , kind='cubic')
        
        total_spctr_Antaress_HE[i] = testF[0]
        
        total_err_Antaress_HE[i] = np.sqrt(testF[1][0])
        
        testD = bind.resampling(ref_wave_Antaress, preliminary_lamda_HE[i], tota_telluric_HE[i] , kind='cubic')
        
        total_telluric_HE[i] = testD
        
        testX = bind.resampling(ref_wave_Rassine_HE, preliminary_lamda_HE[i], tota_telluric_HE[i], kind='cubic')
        
        rassine_total_telluric_HE[i] = testX


else:
    
    #Making the bins 
    min_lamda = max(tota_lamda[:, 0])
    max_lamda = min(tota_lamda[:, -1])
    
    ref_wave = np.linspace(min_lamda, max_lamda, len(tota_lamda[0]))

    #Making the bins for Antaress
    preliminary_lamda=np.zeros((tota_lamda.shape[0], tota_lamda.shape[1]+1))
    
    for i in range(len(preliminary_lamda)):
        preliminary_lamda[i] = np.append(tota_lamda[i], tota_lamda[i][-1]+np.diff(tota_lamda[i])[-1])
    
    ref_wave_Antaress = np.append(ref_wave, ref_wave[-1]+np.diff(ref_wave)[-1])

    #Making the bins for Rassine (telluric model on rassine wavelength solution)
    #Since all wavelength solutions of Rassine are the same we can set it to any of them
    ref_wave_Rassine = np.append(rassine_total_lamda[0], rassine_total_lamda[0][-1] + np.diff(rassine_total_lamda[0])[-1])

    #Initializing variables
    total_lamda = np.zeros(tota_lamda.shape)

    total_norm_spctr = np.zeros(tota_norm_spctr.shape)

    total_norm_spctr_Antaress = np.zeros(tota_norm_spctr.shape)
        
    total_norm_err = np.zeros(tota_norm_err.shape)
        
    total_norm_err_Antaress = np.zeros(tota_norm_err.shape)

    total_spctr = np.zeros(tota_spctr.shape)
    
    total_spctr_Antaress = np.zeros(tota_spctr.shape)

    total_err = np.zeros(tota_err.shape)
    
    total_err_Antaress = np.zeros(tota_err.shape)
    
    total_telluric = np.zeros(tota_telluric.shape)
    
    rassine_total_telluric = np.zeros(rassine_total_lamda.shape)
        
    for i in range(len(tota_lamda)):
        
        interpol_s = interp1d(tota_lamda[i], tota_norm_spctr[i])
        total_norm_spctr[i] = interpol_s(ref_wave)
        
        interpol_err = interp1d(tota_lamda[i], tota_norm_err[i]**2)
        total_norm_err[i] = np.sqrt(interpol_err(ref_wave))
        
        test3 = bind.resampling(ref_wave_Antaress, preliminary_lamda[i], tota_norm_spctr[i], cov = np.array([tota_norm_err[i]**2]) , kind='cubic')
        
        total_norm_spctr_Antaress[i] = test3[0]
        
        total_norm_err_Antaress[i] = np.sqrt(test3[1][0])
                    
        test4 = bind.resampling(ref_wave_Antaress, preliminary_lamda[i], tota_spctr[i], cov = np.array([tota_err[i]**2]) , kind='cubic')
        
        total_spctr_Antaress[i] = test4[0]
        
        total_err_Antaress[i] = np.sqrt(test4[1][0])
        
        test5 = bind.resampling(ref_wave_Antaress, preliminary_lamda[i], tota_telluric[i], kind='cubic')
        
        total_telluric[i] = test5
        
        test6 = bind.resampling(ref_wave_Rassine, preliminary_lamda[i], tota_telluric[i], kind='cubic')
        
        rassine_total_telluric[i] = test6
        
        interpol_spctr = interp1d(tota_lamda[i], tota_spctr[i])
        total_spctr[i] = interpol_spctr(ref_wave)

        interpol_error = interp1d(tota_lamda[i], tota_err[i]**2)
        total_err[i] = np.sqrt(interpol_error(ref_wave))

        total_lamda[i] = ref_wave

# Planetary signal simulation

In [ ]:
###################################################################################
################# ADAPTED FROM WILLIAM DETHIER'S CODE #############################
###################################################################################

In [ ]:
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')

## Defining the parameters for our simulated planet trace

In [ ]:
# Helium density
density = 5e14

# He temperature -- thermal broadening
temperature = 10000

# Atmospheric rotation velocity -- non-thermal broadening
vsini = None

# Reference medium for the wavelengths
medium= "VACUUM"

# Instrumental resolving power e.g. R_pow = 70000
# For non thermal broadening
# see spectrum_he() in spectrum_model.py to know where to write the argument

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

#Radial velocity offset due to movement of the planet - set to 0 for simplicity
RV_offset_He = 0

# Microturbulence
# for non thermal broadening
# micro_t = 1

# You can change the damping to have Voigt profiles, by default you have Gaussian profiles
# Set to value greater than 0
# d_damping = 0

## Simulating on one spectrum to see what it looks like

In [ ]:
reload(AF)

#Random index of the spectrum to plot.
idx = 10

#Defining a variable to save or not the results.
save = False

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #Making a theoretical spectra with a planetary absorption signal to see what it looks like.
    
    theoretical_spctr = spec_mod.spectrum_he(rassine_total_lamda_HE[idx], rassine_total_norm_spctr_HE[idx], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HE)
    theoretical_err = spec_mod.spectrum_he(rassine_total_lamda_HE[idx], rassine_total_norm_err_HE[idx], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HE)

    #Plotting the current spectra with and without a planetary signal.
    plt.figure(figsize=[15, 8])
    plt.errorbar(rassine_total_lamda_HE[idx], rassine_total_norm_spctr_HE[idx], yerr = rassine_total_norm_err_HE[idx], fmt='b.', label='Without Planet')
    plt.errorbar(rassine_total_lamda_HE[idx], theoretical_spctr, yerr = theoretical_err, fmt='r.', label='With Planet')
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    plt.xlim([10820, 10840])
    plt.legend()
    plt.title('Normalized TC Spectrum with and without planetary absorption for HE')
    if save:
        plt.savefig('/Users/samsonmercier/Desktop/Planet.pdf')
    plt.show()
    
    #Plotting the absorption spectrum of the fake planet.
    plt.figure(figsize=[10, 6])
    plt.errorbar(rassine_total_lamda_HE[idx], (rassine_total_norm_spctr_HE[idx]-theoretical_spctr)*100, fmt='b.')
    plt.xlabel('Wavelength ($\AA$)')
    plt.title('Absorption Spectrum')
    plt.ylabel('Excess absorption (%)')
    plt.xlim([10820, 10840])
    plt.show()

#If there is one mode of observation
else:
    
    #Making a theoretical spectra with a planetary absorption signal to see what it looks like.

    theoretical_spctr = spec_mod.spectrum_he(rassine_total_lamda[idx], rassine_total_norm_spctr[idx], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power)
    theoretical_err = spec_mod.spectrum_he(rassine_total_lamda[idx], rassine_total_norm_err[idx], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power)

    #Plotting the current spectra with and without a planetary signal.
    plt.figure(figsize=[10, 6])
    plt.errorbar(rassine_total_lamda[idx], rassine_total_norm_spctr[idx], yerr = rassine_total_norm_err[idx], fmt='b.', label='Out-of-transit')
    plt.errorbar(rassine_total_lamda[idx], theoretical_spctr, yerr = theoretical_err, fmt='r.', label='In-transit')
    plt.xlabel('Wavelength ($\AA$)', fontsize=14)
    plt.ylabel('Normalized Flux', fontsize=14)
    plt.xlim([10820, 10840])
    plt.legend(fontsize=14)
    plt.tick_params(axis='both', labelsize=14)
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Planet.pdf')
    plt.show()
    
    #Plotting the absorption spectrum of the fake planet.
    plt.figure(figsize=[10, 6])
    plt.errorbar(rassine_total_lamda[idx], (1-theoretical_spctr/rassine_total_norm_spctr[idx])*100, fmt='b.')
    plt.xlabel('Wavelength ($\AA$)', fontsize=14)
    plt.ylabel('Excess absorption (%)', fontsize=14)
    plt.tick_params(axis='both', labelsize=14)
    plt.xlim([10820, 10840])
    plt.savefig('/Users/samsonmercier/Downloads/t1.pdf')
    plt.show()

## Defining in-transit and out-of-transit parts of the timeseries

In [ ]:
reload(AF)

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):    
    
    #Data - HA
    spctr_to_use = rassine_total_norm_spctr_HA
    err_to_use = rassine_total_norm_err_HA
    time_to_use = date_HA
    
    #Defining the bounds of transit
    low_bound_HA = int(len(time_to_use)/4)
    up_bound_HA = int(3*len(time_to_use)/4)

    #Making the spectra
    #Before transit
    out_transit_spctr_HA_1 = AF.bound(time_to_use[0], time_to_use[low_bound_HA], time_to_use, spctr_to_use)
    out_transit_err_HA_1 = AF.bound(time_to_use[0], time_to_use[low_bound_HA], time_to_use, err_to_use)
    #In transit
    in_transit_spctr_HA = AF.bound(time_to_use[low_bound_HA], time_to_use[up_bound_HA], time_to_use, spctr_to_use)
    in_transit_err_HA = AF.bound(time_to_use[low_bound_HA], time_to_use[up_bound_HA], time_to_use, err_to_use)
    #After transit
    out_transit_spctr_HA_2 = AF.bound(time_to_use[up_bound_HA], time_to_use[-1], time_to_use, spctr_to_use)
    out_transit_err_HA_2 = AF.bound(time_to_use[up_bound_HA], time_to_use[-1], time_to_use, err_to_use)
    #Out-of-transit
    out_transit_spctr_HA = np.concatenate((out_transit_spctr_HA_1, out_transit_spctr_HA_2)) 
    out_transit_err_HA = np.concatenate((out_transit_err_HA_1, out_transit_err_HA_2)) 

    #Making the timestamps in transit -- for plotting purposes later
    date_in_transit_HA = AF.bound(time_to_use[low_bound_HA], time_to_use[up_bound_HA], time_to_use, time_to_use)

    #Data - HE
    spctr_to_use = rassine_total_norm_spctr_HE
    err_to_use = rassine_total_norm_err_HE
    time_to_use = date_HE

    #Defining the bounds of transit
    low_bound_HE = int(len(time_to_use)/4)
    up_bound_HE = int(3*len(time_to_use)/4)

    #Before transit
    out_transit_spctr_HE_1 = AF.bound(time_to_use[0], time_to_use[low_bound_HE], time_to_use, spctr_to_use)
    out_transit_err_HE_1 = AF.bound(time_to_use[0], time_to_use[low_bound_HE], time_to_use, err_to_use)
    #In transit
    in_transit_spctr_HE = AF.bound(time_to_use[low_bound_HE], time_to_use[up_bound_HE], time_to_use, spctr_to_use)
    in_transit_err_HE = AF.bound(time_to_use[low_bound_HE], time_to_use[up_bound_HE], time_to_use, err_to_use)
    #After transit
    out_transit_spctr_HE_2 = AF.bound(time_to_use[up_bound_HE], time_to_use[-1], time_to_use, spctr_to_use)
    out_transit_err_HE_2 = AF.bound(time_to_use[up_bound_HE], time_to_use[-1], time_to_use, err_to_use)
    #Out-of-transit
    out_transit_spctr_HE = np.concatenate((out_transit_spctr_HE_1, out_transit_spctr_HE_2)) 
    out_transit_err_HE = np.concatenate((out_transit_err_HE_1, out_transit_err_HE_2)) 
  
    #Making the timestamps in transit -- for plotting purposes later
    date_in_transit_HE = AF.bound(time_to_use[low_bound_HE], time_to_use[up_bound_HE], time_to_use, time_to_use)

#If there is one mode of observation
else:    
    #Defining the bounds of transit
    low_bound = int(len(date)/4)
    up_bound = int(3*len(date)/4)
 
    #Data 
    spctr_to_use = rassine_total_norm_spctr
    err_to_use = rassine_total_norm_err
    time_to_use = date

    #Making the spectra
    #Before transit
    out_transit_spctr_1 = AF.bound(time_to_use[0], time_to_use[low_bound], time_to_use, spctr_to_use)
    out_transit_err_1 = AF.bound(time_to_use[0], time_to_use[low_bound], time_to_use, err_to_use)
    #In transitc
    in_transit_spctr = AF.bound(time_to_use[low_bound], time_to_use[up_bound], time_to_use, spctr_to_use)
    in_transit_err = AF.bound(time_to_use[low_bound], time_to_use[up_bound], time_to_use, err_to_use)
    #After transit
    out_transit_spctr_2 = AF.bound(date[up_bound], time_to_use[-1], time_to_use, spctr_to_use)
    out_transit_err_2 = AF.bound(date[up_bound], time_to_use[-1], time_to_use, err_to_use)
    #Out-of-transit
    out_transit_spctr = np.concatenate((out_transit_spctr_1, out_transit_spctr_2)) 
    out_transit_err = np.concatenate((out_transit_err_1, out_transit_err_2)) 

    #Making the timestamps in transit -- for plotting purposes later
    date_in_transit = AF.bound(time_to_use[low_bound], time_to_use[up_bound], time_to_use, time_to_use)


## Adding the planetary signal to all in-transit spectra

In [ ]:
reload(AF)

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #Initiating arrays to contain the in-transit spectra with a planetary signal and their corresponding errors.
    planet_in_transit_spctr_HA = np.ones(in_transit_spctr_HA.shape)
    planet_in_transit_err_HA = np.ones(in_transit_err_HA.shape)
    
    planet_in_transit_spctr_HE = np.ones(in_transit_spctr_HE.shape)
    planet_in_transit_err_HE = np.ones(in_transit_err_HE.shape)
    
    #Populating the above defined arrays.
    wav_to_use = rassine_total_lamda_HA[0]
    for i in range(len(in_transit_spctr_HA)):

        planet_in_transit_spctr_HA[i] = spec_mod.spectrum_he(wav_to_use, in_transit_spctr_HA[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HA)
        planet_in_transit_err_HA[i] = spec_mod.spectrum_he(wav_to_use, in_transit_err_HA[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HA)

    wav_to_use = rassine_total_lamda_HE[0]
    for i in range(len(in_transit_spctr_HE)):

        planet_in_transit_spctr_HE[i] = spec_mod.spectrum_he(wav_to_use, in_transit_spctr_HE[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HE)
        planet_in_transit_err_HE[i] = spec_mod.spectrum_he(wav_to_use, in_transit_err_HE[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HE)

#If there is one mode of observation    
else:
    #Initiating arrays to contain the in-transit spectra with a planetary signal and their corresponding errors.
    planet_in_transit_spctr = np.ones(in_transit_spctr.shape)
    planet_in_transit_err = np.ones(in_transit_err.shape)
    
    wav_to_use = rassine_total_lamda[0]
    
    #Populating the above defined arrays.
    for i in range(len(in_transit_spctr)):

        planet_in_transit_spctr[i] = spec_mod.spectrum_he(wav_to_use, in_transit_spctr[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power)
        planet_in_transit_err[i] = spec_mod.spectrum_he(wav_to_use, in_transit_err[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power)



## Getting the master spectrum in and out of transit

In [ ]:
reload(AF)

using_Rassine = True

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #Making the master spectra, in and out of transit.
    if using_Rassine:
        #In-transit
        master_planet_intransit_HA = spec_mod.spectrum_he(rassine_master_lamda_HA, rassine_master_spctr_HA, temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HA)
        master_err_intransit_HA = np.zeros(master_planet_intransit_HA.shape)
        
        master_planet_intransit_HE = spec_mod.spectrum_he(rassine_master_lamda_HE, rassine_master_spctr_HE, temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HE)
        master_err_intransit_HE = np.zeros(master_planet_intransit_HE.shape)

        #Out-of-transit 
        master_outtransit_HA = rassine_master_spctr_HA
        master_err_outtransit_HA = np.zeros(rassine_master_spctr_HA.shape)

        master_outtransit_HE = rassine_master_spctr_HE
        master_err_outtransit_HE = np.zeros(rassine_master_spctr_HE.shape)

    else:
        #In-transit
        master_planet_intransit_HA = np.average(planet_in_transit_spctr_HA, axis=0, weights = 1/planet_in_transit_err_HA**2)
        master_err_intransit_HA = 1/np.sqrt(np.sum(1/planet_in_transit_err_HA**2, axis=0))

        master_planet_intransit_HE = np.average(planet_in_transit_spctr_HE, axis=0, weights = 1/planet_in_transit_err_HE**2)
        master_err_intransit_HE = 1/np.sqrt(np.sum(1/planet_in_transit_err_HE**2, axis=0))

        #Out-of-transit
        master_outtransit_HA = np.average(out_transit_spctr_HA, axis=0, weights = 1/out_transit_err_HA**2)
        master_err_outtransit_HA = 1/np.sqrt(np.sum(1/out_transit_err_HA**2, axis=0))

        master_outtransit_HE = np.average(out_transit_spctr_HE, axis=0, weights = 1/out_transit_err_HE**2)
        master_err_outtransit_HE = 1/np.sqrt(np.sum(1/out_transit_err_HE**2, axis=0))
    
#If there is one mode of observation
else:
    #Making the master spectra, in and out of transit.
    if using_Rassine:
        #In-transit
        master_planet_intransit = spec_mod.spectrum_he(rassine_master_lamda, rassine_master_spctr, temperature, density, vsini, medium, RV_offset_He, R_pow = R_power)
        master_err_intransit = np.zeros(master_planet_intransit.shape)
        
        #Out-of-transit 
        master_outtransit = rassine_master_spctr
        master_err_outtransit = np.zeros(rassine_master_spctr.shape)
    else:
        #In-transit
        master_planet_intransit = np.average(planet_in_transit_spctr, axis=0, weights = 1/planet_in_transit_err**2)
        master_err_intransit = 1/np.sqrt(np.sum(1/planet_in_transit_err**2, axis=0))

        #Out-of-transit
        master_outtransit = np.average(out_transit_spctr, axis=0, weights = 1/out_transit_err**2)
        master_err_outtransit = 1/np.sqrt(np.sum(1/out_transit_err**2, axis=0))
    

In [ ]:
reload(AF)
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #Plotting the master spectra 
    wav_to_use = rassine_master_lamda_HA
    plt.figure(figsize=[10, 6])
    plt.errorbar(wav_to_use, master_outtransit_HA, yerr = master_err_outtransit_HA, fmt='b.', label='Out-of-transit')
    plt.errorbar(wav_to_use, master_planet_intransit_HA, yerr = master_err_intransit_HA, fmt='r.', label='In-transit')
    plt.legend()
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    plt.title('Master Spectrum In and Out of Transit for HA')
    plt.xlim([10820, 10840])
    plt.show()

    wav_to_use = rassine_master_lamda_HE
    plt.figure(figsize=[10, 6])
    plt.errorbar(wav_to_use, master_outtransit_HE, yerr = master_err_outtransit_HE, fmt='b.', label='Out-of-transit')
    plt.errorbar(wav_to_use, master_planet_intransit_HE, yerr = master_err_intransit_HE, fmt='r.', label='In-transit')
    plt.legend()
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    plt.title('Master Spectrum In and Out of Transit for HE')
    plt.xlim([10820, 10840])
    plt.show()

#If there is one mode of observation
else:
    wav_to_use = rassine_master_lamda
    #Plotting the master spectra
    plt.figure(figsize=[10, 6])
    plt.errorbar(wav_to_use, master_outtransit, yerr = master_err_outtransit, fmt='b.', label='Out-of-transit')
    plt.errorbar(wav_to_use, master_planet_intransit, yerr = master_err_intransit, fmt='r.', label='In-transit')
    plt.legend()
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    plt.title('Master Spectrum In and Out of Transit')
    plt.xlim([10820, 10840])
    plt.show()


## Making the absorption spectra

In [ ]:
reload(AF)

##Defining the difference between the in-transit and master out-of-transit spectra to get 
#a time series of the planet's spectrum
offset = 1

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #Initiating arrays to contain the values and error of the planetary absorption spectrum.
    diff_HA = np.ones(planet_in_transit_spctr_HA.shape)
    diff_HA_err = np.ones(planet_in_transit_err_HA.shape)
    
    #Populating the above defined arrays.
    #We add an offset of 1 for later fitting purposes.
    for i in range(len(planet_in_transit_spctr_HA)):
        diff_HA[i] = offset+(master_outtransit_HA - planet_in_transit_spctr_HA[i])/master_outtransit_HA
        diff_HA_err[i] = np.sqrt((planet_in_transit_err_HA[i]/master_outtransit_HA)**2 + ((planet_in_transit_spctr_HA[i] * master_err_outtransit_HA)/(master_outtransit_HA**2))**2)

    diff_HE = np.ones(planet_in_transit_spctr_HE.shape)
    diff_HE_err = np.ones(planet_in_transit_err_HE.shape)
    for i in range(len(planet_in_transit_spctr_HE)):
        diff_HE[i] = offset+(master_outtransit_HE - planet_in_transit_spctr_HE[i])/master_outtransit_HE
        diff_HE_err[i] = np.sqrt((planet_in_transit_err_HE[i]/master_outtransit_HE)**2 + ((planet_in_transit_spctr_HE[i] * master_err_outtransit_HE)/(master_outtransit_HE**2))**2)

#If there is one mode of observation
else:
    
    #Initiating arrays to contain the values and error of the planetary absorption spectrum.
    diff = np.ones(planet_in_transit_spctr.shape)
    diff_err = np.ones(planet_in_transit_err.shape)
    
    #Populating the above defined arrays.
    #We add an offset of 1 for later fitting purposes.
    for i in range(len(planet_in_transit_spctr)):
        diff[i] = offset+(master_outtransit - planet_in_transit_spctr[i])/master_outtransit
        diff_err[i] = np.sqrt((planet_in_transit_err[i]/master_outtransit)**2 + ((planet_in_transit_spctr[i] * master_err_outtransit)/(master_outtransit**2))**2)



## Fitting the absorption spectra

In [ ]:
reload(AF)

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    base_ref_wave_HA = rassine_total_lamda_HA[0]
    
    base_ref_wave_HE = rassine_total_lamda_HE[0]
    #Re-defining the absorption function
    #We fix the stellar velocity because the Sun's velocity is well known.
    #We fit for the atmospheric He column density and the temperature of the atmosphere.
    def base_absorption_function_HA(x, temp, density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_HA[0].shape), temp, density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HA)

    def base_absorption_function_HE(x, temp, density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_HE[0].shape), temp, density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HE)

    #Creating arrays to store the best-fit parameters and the error on them
    base_param_HA = np.zeros((len(diff_HA), 3))
    base_err_HA = np.zeros((len(diff_HA), 3))

    base_param_HE = np.zeros((len(diff_HE), 3))
    base_err_HE = np.zeros((len(diff_HE), 3))

    #Iterating over the time series to get a value of the retrieved parameters for each spectrum
    for i in range(len(diff_HA)):
        
        #Guess on the initial parameters
        guess = [14000, 1e14, np.median(diff_HA[i])]

        #Chi-squared minimization routine
        params, cov = curve_fit(base_absorption_function_HA, base_ref_wave_HA, diff_HA[i], sigma = diff_HA_err[i], p0=guess)

        #Storing the best-fit parameters and errors
        base_param_HA[i] = params
        base_err_HA[i] = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        print(params)
        model = base_absorption_function_HA(base_ref_wave_HA, params[0], params[1], params[2])

        plt.figure(figsize=[10, 6])
        plt.errorbar(base_ref_wave_HA, 100*(diff_HA[i]-offset), yerr=100*diff_HA_err[i], fmt='b.', label='Simulated data')
        plt.plot(base_ref_wave_HA, 100*(base_absorption_function_HA(base_ref_wave_HA, temperature, -density, params[2])-offset), color='orange', label='Expected result')
        plt.plot(base_ref_wave_HA, 100*(model-offset), 'r', label='Best-fit')
        plt.xlim([10825, 10840])
        plt.xlabel('Wavelength ($\AA$)')
        plt.ylabel('Absorption (%)')
        plt.ylim(-2, 7)
        plt.legend()
        plt.show()
        
    for i in range(len(diff_HE)):
        
        #Guess on the initial parameters
        guess = [16000, 1e14, np.median(diff_HE[i])]
        
        #Chi-squared minimization routine
        params, cov = curve_fit(base_absorption_function_HE, base_ref_wave_HE, diff_HE[i], sigma=diff_HE_err[i], p0=guess)

        #Storing the best-fit parameters and errors
        base_param_HE[i] = params
        base_err_HE[i] = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        print(params)
        model = base_absorption_function_HE(ref_wave_HE, params[0], params[1], params[2])

        plt.figure(figsize=[10, 6])
        plt.errorbar(base_ref_wave_HE, 100*(diff_HE[i]-offset), yerr = 100*diff_HE_err[i], fmt='b.', label='Simulated data')
        plt.plot(base_ref_wave_HE, 100*(base_absorption_function_HE(base_ref_wave_HE, temperature, -density, params[2])-offset), color='orange', label='Expected result')
        plt.plot(base_ref_wave_HE, 100*(model-offset), 'k', label='Best-fit')
        plt.xlim([10825, 10840])
        plt.xlabel('Wavelength ($\AA$)')
        plt.ylabel('Absorption (%)')
        plt.ylim(-2, 7)
        plt.legend()
        plt.show()

#If there is one mode of observation
else:
    
    base_ref_wave = rassine_total_lamda[0]
    
    #Re-defining the absorption function
    #We fix the stellar velocity because the Sun's velocity is well known.
    #We fit for the atmospheric He column density and the temperature of the atmosphere.
    def base_absorption_function(x, temp, density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff[0].shape), temp, density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power)

    #Creating arrays to store the best-fit parameters and the error on them
    base_param = np.zeros((len(diff), 3))
    base_err = np.zeros((len(diff), 3))
    
   #Iterating over the time series to get a value of the retrieved parameters for each spectrum
    for i in range(len(diff)):
        
        guess = [16000, -3e14, np.median(diff[i])]

        #Chi-squared minimization routine
        params, cov = curve_fit(base_absorption_function, base_ref_wave, diff[i], sigma=diff_err[i], p0=guess)
        print(diff_err[i])
        #Storing the best-fit parameters and errors
        base_param[i] = params
        base_err[i] = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        print(params)
        model = base_absorption_function(base_ref_wave, params[0], params[1], params[2])

        plt.figure(figsize=[10, 6])
        plt.errorbar(base_ref_wave, 100*(diff[i]-offset), yerr=100*diff_err[i], fmt='b.', label='Simulated data')
        plt.plot(base_ref_wave, 100*(base_absorption_function(base_ref_wave, temperature, -density, params[2])-offset), color='orange', label='Expected result')
        plt.plot(base_ref_wave, 100*(model-offset), 'r', label='Best-fit')
        plt.xlim([10825, 10840])
        plt.xlabel('Wavelength ($\AA$)')
        plt.ylabel('Absorption (%)')
        plt.legend()
        plt.show()


#For fitting purposes, we needed to have negative densities, so we revert to positive values.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    base_param_HA[:, 1] = -base_param_HA[:, 1]
    base_param_HE[:, 1] = -base_param_HE[:, 1]
#For fitting purposes, we needed to have negative densities, so we revert to positive values.
else:    
    base_param[:, 1] = -base_param[:, 1]



## Sanity check fit on the master absorption spectrum

In [ ]:

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
        
    master_base_ref_wave_HA = rassine_master_lamda_HA
    
    diff_master_HA = offset+(master_outtransit_HA - master_planet_intransit_HA)/master_outtransit_HA

    diff_master_err_HA = np.sqrt((master_err_intransit_HA/master_outtransit_HA)**2 + ((master_planet_intransit_HA * master_err_outtransit_HA)/(master_outtransit_HA**2))**2)

    def master_base_absorption_function_HA(x, temp, density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_master_HA.shape), temp, density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HA)

    guess = [16000, 1e14, np.median(diff_master_HA)]

    #Chi-squared minimization routine
    if using_Rassine: #Because we don't have the error on the master spectrum with Rassine
        params, cov = curve_fit(master_base_absorption_function_HA, master_base_ref_wave_HA, diff_master_HA, p0=guess)
    else:
        params, cov = curve_fit(master_base_absorption_function_HA, master_base_ref_wave_HA, diff_master_HA, sigma=diff_master_err_HA, p0=guess)
 
    #Storing the best-fit parameters and errors
    base_param_master_HA = params
    base_err_master_HA = np.sqrt(np.diag(cov))

    #Plotting the in-transit planetary spectra and their corresponding model
    print(params)
    plt.figure(figsize=[10, 6])
    model = master_base_absorption_function_HA(master_base_ref_wave_HA, params[0], params[1], params[2])
    plt.errorbar(master_base_ref_wave_HA, 100*(diff_master_HA-offset), 100*diff_master_err_HA, fmt='b.', label='Simulated data')
    plt.plot(master_base_ref_wave_HA, 100*(master_base_absorption_function_HA(master_base_ref_wave_HA, temperature, -density, params[2])-offset), color='orange', label='Expected result')
    plt.plot(master_base_ref_wave_HA, 100*(model-offset), 'r', label='Best-fit')
    plt.xlim([10825, 10840])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Absorption (%)')
    plt.legend()
    plt.show()

    master_base_ref_wave_HE = rassine_master_lamda_HE
    
    diff_master_HE = offset+(master_outtransit_HE - master_planet_intransit_HE)/master_outtransit_HE

    diff_master_err_HE = np.sqrt((master_err_intransit_HE/master_outtransit_HE)**2 + ((master_planet_intransit_HE * master_err_outtransit_HE)/(master_outtransit_HE**2))**2)

    def master_base_absorption_function_HE(x, temp, density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_master_HE.shape), temp, density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HE)

    guess = [16000, 1e14, np.median(diff_master_HE)]

    #Chi-squared minimization routine
    if using_Rassine: #Because we don't have the error on the master spectrum with Rassine
        params, cov = curve_fit(master_base_absorption_function_HE, master_base_ref_wave_HE, diff_master_HE, p0=guess)
    else:
        params, cov = curve_fit(master_base_absorption_function_HE, master_base_ref_wave_HE, diff_master_HE, sigma=diff_master_err_HE, p0=guess)

    #Storing the best-fit parameters and errors
    base_param_master_HE = params
    base_err_master_HE = np.sqrt(np.diag(cov))

    #Plotting the in-transit planetary spectra and their corresponding model
    print(params)
    plt.figure(figsize=[10, 6])
    model = master_base_absorption_function_HE(master_base_ref_wave_HE, params[0], params[1], params[2])
    plt.errorbar(master_base_ref_wave_HE, 100*(diff_master_HE-offset), 100*diff_master_err_HE, fmt='b.', label='Simulated data')
    plt.plot(master_base_ref_wave_HE, 100*(master_base_absorption_function_HE(master_base_ref_wave_HE, temperature, -density, params[2])-offset), color='orange', label='Expected result')
    plt.plot(master_base_ref_wave_HE, 100*(model-offset), 'k', label='Best-fit')
    plt.xlim([10825, 10840])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Absorption (%)')
    plt.legend()
    plt.show()
    
    
else:
    
    master_base_ref_wave = rassine_master_lamda
    
    diff_master = offset+(master_outtransit - master_planet_intransit)/master_outtransit

    diff_master_err = np.sqrt((master_err_intransit/master_outtransit)**2 + ((master_planet_intransit * master_err_outtransit)/(master_outtransit**2))**2)

    def master_base_absorption_function(x, temp, density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_master.shape), temp, density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power)

    guess = [16000, 1e14, np.median(diff_master)]

    #Chi-squared minimization routine
    if using_Rassine: #Because we don't have the error on the master spectrum with Rassine
        params, cov = curve_fit(master_base_absorption_function, master_base_ref_wave, diff_master, p0=guess)
    else:
        params, cov = curve_fit(master_base_absorption_function, master_base_ref_wave, diff_master, sigma=diff_master_err, p0=guess)
    #Storing the best-fit parameters and errors
    base_param_master = params
    base_err_master = np.sqrt(np.diag(cov))

    #Plotting the in-transit planetary spectra and their corresponding model
    print(params)
    model = master_base_absorption_function(master_base_ref_wave, params[0], params[1], params[2])

    plt.figure(figsize=[10, 6])
    plt.errorbar(master_base_ref_wave, 100*(diff_master-offset), 100*diff_master_err, fmt='b.', label='Simulated data')
    plt.plot(master_base_ref_wave, 100*(master_base_absorption_function(master_base_ref_wave, temperature, -density, params[2])-offset), color='orange', label='Expected result')
    plt.plot(master_base_ref_wave, 100*(model-offset), 'r', label='Best-fit')
    plt.xlim([10825, 10840])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Absorption (%)')
    plt.legend()
    plt.show()


## Plotting retrieved parameters

In [ ]:
reload(AF)

#Defining a variable to save or not the results.
save = True

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=[18, 12])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HA = np.average(base_param_HA[:, 0], weights=1/base_err_HA[:, 0]**2)
    weighted_std_T_HA = np.sqrt(np.average((base_param_HA[:, 0] - weighted_avg_T_HA)**2, weights=1/base_err_HA[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit_HA, base_param_HA[:, 0], yerr = base_err_HA[:, 0], fmt='.', label='Retrieved')
    ax1.set_xlabel('Time (MJD)')
    ax1.set_ylabel('Temperature (K)')
    ax1.set_title('Best-fit temperature for HA')
    ax1.axhline(temperature, color='k', label='Expected')
    ax1.axhline(weighted_avg_T_HA, color='r', label='Average')
    ax1.axhline(base_param_master_HA[0], color='b', label='Master')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-weighted_std_T_HA, y2 = weighted_avg_T_HA+weighted_std_T_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-2*weighted_std_T_HA, y2 = weighted_avg_T_HA+2*weighted_std_T_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-3*weighted_std_T_HA, y2 = weighted_avg_T_HA+3*weighted_std_T_HA, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(framealpha=0.8)
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HE = np.average(base_param_HE[:, 0], weights=1/base_err_HE[:, 0]**2)
    weighted_std_T_HE = np.sqrt(np.average((base_param_HE[:, 0] - weighted_avg_T_HE)**2, weights=1/base_err_HE[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax2.errorbar(date_in_transit_HE, base_param_HE[:, 0], yerr = base_err_HE[:, 0], fmt='.', label='Retrieved')
    ax2.set_xlabel('Time (MJD)')
    ax2.set_ylabel('Temperature (K)')
    ax2.set_title('Best-fit temperature for HE')
    ax2.axhline(temperature, color='k', label='Expected')
    ax2.axhline(weighted_avg_T_HE, color='r', label='Average')
    ax2.axhline(base_param_master_HE[0], color='b', label='Master')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-weighted_std_T_HE, y2 = weighted_avg_T_HE+weighted_std_T_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-2*weighted_std_T_HE, y2 = weighted_avg_T_HE+2*weighted_std_T_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-3*weighted_std_T_HE, y2 = weighted_avg_T_HE+3*weighted_std_T_HE, alpha=0.2, color='orange', label='3 $\sigma$')
    
    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HA = np.average(base_param_HA[:, 1], weights=1/base_err_HA[:, 1]**2)
    weighted_std_rho_HA = np.sqrt(np.average((base_param_HA[:, 1] - weighted_avg_rho_HA)**2, weights=1/base_err_HA[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax3.errorbar(date_in_transit_HA, base_param_HA[:, 1], yerr = base_err_HA[:, 1], fmt='.', label='Retrieved')
    ax3.set_xlabel('Time (MJD)')
    ax3.set_ylabel('Density ($m^{-3}$)')
    ax3.set_title('Best-fit column density for HA')
    ax3.axhline(density, color='k', label='Expected')
    ax3.axhline(weighted_avg_rho_HA, color='r', label='Average')
    ax3.axhline(-base_param_master_HA[1], color='b', label='Master')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-weighted_std_rho_HA, y2 = weighted_avg_rho_HA+weighted_std_rho_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-2*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+2*weighted_std_rho_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-3*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+3*weighted_std_rho_HA, alpha=0.2, color='orange', label='3 $\sigma$')

    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HE = np.average(base_param_HE[:, 1], weights=1/base_err_HE[:, 1]**2)
    weighted_std_rho_HE = np.sqrt(np.average((base_param_HE[:, 1] - weighted_avg_rho_HE)**2, weights=1/base_err_HE[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax4.errorbar(date_in_transit_HE, base_param_HE[:, 1], yerr = base_err_HE[:, 1], fmt='.', label='Retrieved ')
    ax4.set_xlabel('Time (MJD)')
    ax4.set_ylabel('Density ($m^{-3}$)')
    ax4.set_title('Best-fit column density for HE')
    ax4.axhline(density, color='k', label='Expected')
    ax4.axhline(weighted_avg_rho_HE, color='r', label='Average')
    ax4.axhline(-base_param_master_HE[1], color='b', label='Master')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-weighted_std_rho_HE, y2 = weighted_avg_rho_HE+weighted_std_rho_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-2*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+2*weighted_std_rho_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-3*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+3*weighted_std_rho_HE, alpha=0.2, color='orange', label='3 $\sigma$')

    ax1.sharey(ax2)
    ax3.sharey(ax4)

    fig.tight_layout()
    if save:
        plt.savefig(save_location+'Retrieved_Params.pdf')
        
    print('HA - Expected T is: '+str(temperature)+' and retrieved T is: %.2f'%weighted_avg_T_HA,' and difference is: %.2f'%np.abs((temperature - weighted_avg_T_HA)/weighted_std_T_HA), ' sigma')
    print('HA - Expected density is: '+str(density/1e14)+' and retrieved density is: %.2f'%(weighted_avg_rho_HA / 1e14),' and difference is: %.2f'%np.abs((density - weighted_avg_rho_HA)/weighted_std_rho_HA), ' sigma')
    print('HE - Expected T is: '+str(temperature)+' and retrieved T is: %.2f'%weighted_avg_T_HE,' and difference is: %.2f'%np.abs((temperature - weighted_avg_T_HE)/weighted_std_T_HE), ' sigma')
    print('HE - Expected density is: '+str(density/1e14)+' and retrieved density is: %.2f'%(weighted_avg_rho_HE / 1e14),' and difference is: %.2f'%np.abs((density - weighted_avg_rho_HE)/weighted_std_rho_HE), ' sigma')


#If there is one mode of observation
else:
    
    fig , (ax1, ax2) = plt.subplots(1, 2, figsize=[14, 8])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T = np.average(base_param[:, 0], weights=1/base_err[:, 0]**2)
    weighted_std_T = np.sqrt(np.average((base_param[:, 0] - weighted_avg_T)**2, weights=1/base_err[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit, base_param[:, 0], yerr = base_err[:, 0], fmt='.', label='Retrieved')
    ax1.set_xlabel('Time (MJD)')
    ax1.set_ylabel('Temperature (K)')
    ax1.set_title('Best-fit temperature')
    ax1.axhline(temperature, color='k', label='Expected')
    ax1.axhline(weighted_avg_T, color='r', label='Average')
    ax1.axhline(base_param_master[0], color='b', label='Master')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-weighted_std_T, y2 = weighted_avg_T+weighted_std_T, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-2*weighted_std_T, y2 = weighted_avg_T+2*weighted_std_T, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-3*weighted_std_T, y2 = weighted_avg_T+3*weighted_std_T, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(framealpha=.7)

    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho = np.average(base_param[:, 1], weights=1/base_err[:, 1]**2)
    weighted_std_rho = np.sqrt(np.average((base_param[:, 1] - weighted_avg_rho)**2, weights=1/base_err[:, 1]**2))
        
    #Plotting the retrieved densities.
    ax2.errorbar(date_in_transit, base_param[:, 1], yerr = base_err[:, 1], fmt='.', label='Retrieved')
    ax2.set_xlabel('Time (MJD)')
    ax2.set_ylabel('Density ($cm^{-2}$)')
    ax2.set_title('Best-fit column density')
    ax2.axhline(density, color='k', label='Expected')
    ax2.axhline(weighted_avg_rho, color='r', label='Average')
    ax2.axhline(-base_param_master[1], color='b', label='Master')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-weighted_std_rho, y2 = weighted_avg_rho+weighted_std_rho, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-2*weighted_std_rho, y2 = weighted_avg_rho+2*weighted_std_rho, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-3*weighted_std_rho, y2 = weighted_avg_rho+3*weighted_std_rho, alpha=0.2, color='orange', label='3 $\sigma$')
    fig.tight_layout()
    if save:
        plt.savefig(save_location+'Retrieved_Params.pdf')
    
    print('Expected T is: '+str(temperature)+' and retrieved T is: %.2f'%weighted_avg_T,' and difference is: %.2f'%np.abs((temperature - weighted_avg_T)/weighted_std_T), ' sigma')
    print('Expected density is: '+str(density/1e14)+' and retrieved density is: %.2f'%(weighted_avg_rho / 1e14),' and difference is: %.2f'%np.abs((density - weighted_avg_rho)/weighted_std_rho), ' sigma')

   

## Plotting periodogram of best-fit parameters

In [ ]:
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    LS_T_HA = LombScargle(date_in_transit_HA, base_param_HA[:, 0], base_err_HA[:, 0]).autopower(nyquist_factor=1)
    LS_T_HE = LombScargle(date_in_transit_HE, base_param_HE[:, 0], base_err_HE[:, 1]).autopower(nyquist_factor=1)
    
    LS_rho_HA = LombScargle(date_in_transit_HA, base_param_HA[:, 1], base_err_HA[:, 0]).autopower(nyquist_factor=1)
    LS_rho_HE = LombScargle(date_in_transit_HE, base_param_HE[:, 1], base_err_HE[:, 1]).autopower(nyquist_factor=1)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=[10, 4])
    ax1.loglog(LS_T_HA[0], LS_T_HA[1])
    ax1.set_xlabel('Frequency (1/d)')
    ax1.set_ylabel('Power')
    ax1.set_title('Temperature Periodogram for HA')
    ax1.axvline(LS_T_HA[0][LS_T_HA[1]==np.max(LS_T_HA[1])])
    
    ax2.loglog(LS_T_HE[0], LS_T_HE[1])
    ax2.set_xlabel('Frequency (1/d)')
    ax2.set_title('Temperature Periodogram for HE')
    ax2.axvline(LS_T_HE[0][LS_T_HE[1]==np.max(LS_T_HE[1])])
    print(LS_T_HA[0][LS_T_HA[1]==np.max(LS_T_HA[1])], LS_T_HE[0][LS_T_HE[1]==np.max(LS_T_HE[1])])
    
    fig.tight_layout()
    plt.show()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=[10, 4])
    ax1.loglog(LS_rho_HA[0], LS_rho_HA[1])
    ax1.set_xlabel('Frequency (1/d)')
    ax1.set_ylabel('Power')
    ax1.set_title('Column Density Periodogram for HA')
    ax1.axvline(LS_rho_HA[0][LS_rho_HA[1]==np.max(LS_rho_HA[1])])
    
    ax2.loglog(LS_rho_HE[0], LS_rho_HE[1])
    ax2.set_xlabel('Frequency (1/d)')
    ax2.set_title('Column Density Periodogram for HE')
    ax2.axvline(LS_rho_HE[0][LS_rho_HE[1]==np.max(LS_rho_HE[1])])
    print(LS_rho_HA[0][LS_rho_HA[1]==np.max(LS_rho_HA[1])], LS_rho_HE[0][LS_rho_HE[1]==np.max(LS_rho_HE[1])])

    fig.tight_layout()
    plt.show()

else:

    LS_T = LombScargle(date_in_transit, base_param[:, 0], base_err[:, 0]).autopower(nyquist_factor=1)
    
    LS_rho = LombScargle(date_in_transit, base_param[:, 1], base_err[:, 1]).autopower(nyquist_factor=1)
    
    plt.loglog(LS_T[0], LS_T[1])
    plt.xlabel('Frequency (1/d)')
    plt.ylabel('Power')
    plt.title('Temperature Periodogram for HA')
    plt.axvline(LS_T[0][LS_T[1]==np.max(LS_T[1])])
    print(LS_T[0][LS_T[1]==np.max(LS_T[1])])
    
    plt.show()
    
    plt.loglog(LS_rho[0], LS_rho[1])
    plt.xlabel('Frequency (1/d)')
    plt.ylabel('Power')
    plt.title('Column Density Periodogram for HA')
    plt.axvline(LS_rho[0][LS_rho[1]==np.max(LS_rho[1])])
    print(LS_rho[0][LS_rho[1]==np.max(LS_rho[1])])

    plt.show()


## Figuring out if there is a sinusoidal trend in the best-fit parameters

In [ ]:
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    omega_rho_HA = LS_rho_HA[0][LS_rho_HA[1]==np.max(LS_rho_HA[1])][0]
    omega_T_HA = LS_T_HA[0][LS_T_HA[1]==np.max(LS_T_HA[1])][0]

    def sinusoid_T_HA(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_T_HA) + B*np.sin(2*np.pi*t*omega_T_HA)

    def sinusoid_rho_HA(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_rho_HA) + B*np.sin(2*np.pi*t*omega_rho_HA)

    base_param_T_HA, base_cov_T_HA =  curve_fit(sinusoid_T_HA, date_in_transit_HA, base_param_HA[:, 0], sigma = base_err_HA[:, 0])
    base_param_rho_HA, base_cov_rho_HA =  curve_fit(sinusoid_rho_HA, date_in_transit_HA, base_param_HA[:, 1], sigma = base_err_HA[:, 1], p0=[5e13, 5e13, 4.6e14])

    omega_rho_HE = LS_rho_HE[0][LS_rho_HE[1]==np.max(LS_rho_HE[1])][0]
    omega_T_HE = LS_T_HE[0][LS_T_HE[1]==np.max(LS_T_HE[1])][0]

    def sinusoid_T_HE(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_T_HE) + B*np.sin(2*np.pi*t*omega_T_HE)

    def sinusoid_rho_HE(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_rho_HE) + B*np.sin(2*np.pi*t*omega_rho_HE)

    base_param_T_HE, base_cov_T_HE =  curve_fit(sinusoid_T_HE, date_in_transit_HE, base_param_HE[:, 0], sigma = base_err_HE[:, 0])
    base_param_rho_HE, base_cov_rho_HE =  curve_fit(sinusoid_rho_HE, date_in_transit_HE, base_param_HE[:, 1], sigma = base_err_HE[:, 1], p0=[5e13, 5e13, 4.6e14])

else:
    omega_rho = LS_rho[0][LS_rho[1]==np.max(LS_rho[1])]
    omega_T = LS_T[0][LS_T[1]==np.max(LS_T[1])]
    
    def sinusoid_T(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_T) + B*np.sin(2*np.pi*t*omega_T)

    def sinusoid_rho(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_rho) + B*np.sin(2*np.pi*t*omega_rho)


    base_param_T, base_cov_T =  curve_fit(sinusoid_T, date_in_transit, base_param[:, 0], sigma = base_err[:, 0])
    base_param_rho, base_cov_rho =  curve_fit(sinusoid_rho, date_in_transit, base_param[:, 1], sigma = base_err[:, 1], p0=[5e13, 5e13, 4.6e14])


In [ ]:
reload(AF)

#Defining a variable to save or not the results.
save = False

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=[18, 12])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HA = np.average(param_HA[:, 0], weights=1/err_HA[:, 0]**2)
    weighted_std_T_HA = np.sqrt(np.average((param_HA[:, 0] - weighted_avg_T_HA)**2, weights=1/err_HA[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit_HA, param_HA[:, 0], yerr = err_HA[:, 0], fmt='.', label='Retrieved')
    ax1.plot(date_in_transit_HA, sinusoid_T_HA(date_in_transit_HA, param_T_HA[0], param_T_HA[1], param_T_HA[2]), 'm', label='Model')
    ax1.set_xlabel('Time (MJD)')
    ax1.set_ylabel('Temperature (K)')
    ax1.set_title('Best-fit temperature for HA')
    ax1.axhline(temperature, color='k', label='Expected')
    ax1.axhline(weighted_avg_T_HA, color='r', label='Average')
    ax1.axhline(param_master_HA[0], color='b', label='Master')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-weighted_std_T_HA, y2 = weighted_avg_T_HA+weighted_std_T_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-2*weighted_std_T_HA, y2 = weighted_avg_T_HA+2*weighted_std_T_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-3*weighted_std_T_HA, y2 = weighted_avg_T_HA+3*weighted_std_T_HA, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(framealpha=0.8)
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HE = np.average(param_HE[:, 0], weights=1/err_HE[:, 0]**2)
    weighted_std_T_HE = np.sqrt(np.average((param_HE[:, 0] - weighted_avg_T_HE)**2, weights=1/err_HE[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax2.errorbar(date_in_transit_HE, param_HE[:, 0], yerr = err_HE[:, 0], fmt='.', label='Retrieved')
    ax2.plot(date_in_transit_HE, sinusoid_T_HE(date_in_transit_HE, param_T_HE[0], param_T_HE[1], param_T_HE[2]), 'm', label='Model')
    ax2.set_xlabel('Time (MJD)')
    ax2.set_ylabel('Temperature (K)')
    ax2.set_title('Best-fit temperature for HE')
    ax2.axhline(temperature, color='k', label='Expected')
    ax2.axhline(weighted_avg_T_HE, color='r', label='Average')
    ax2.axhline(param_master_HE[0], color='b', label='Master')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-weighted_std_T_HE, y2 = weighted_avg_T_HE+weighted_std_T_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-2*weighted_std_T_HE, y2 = weighted_avg_T_HE+2*weighted_std_T_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-3*weighted_std_T_HE, y2 = weighted_avg_T_HE+3*weighted_std_T_HE, alpha=0.2, color='orange', label='3 $\sigma$')
    
    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HA = np.average(param_HA[:, 1], weights=1/err_HA[:, 1]**2)
    weighted_std_rho_HA = np.sqrt(np.average((param_HA[:, 1] - weighted_avg_rho_HA)**2, weights=1/err_HA[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax3.errorbar(date_in_transit_HA, param_HA[:, 1], yerr = err_HA[:, 1], fmt='.', label='Retrieved')
    ax3.plot(date_in_transit_HA, sinusoid_rho_HA(date_in_transit_HA, param_rho_HA[0], param_rho_HA[1], param_rho_HA[2]), 'm', label='Model')
    ax3.set_xlabel('Time (MJD)')
    ax3.set_ylabel('Density ($m^{-3}$)')
    ax3.set_title('Best-fit column density for HA')
    ax3.axhline(density, color='k', label='Expected')
    ax3.axhline(weighted_avg_rho_HA, color='r', label='Average')
    ax3.axhline(-param_master_HA[1], color='b', label='Master')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-weighted_std_rho_HA, y2 = weighted_avg_rho_HA+weighted_std_rho_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-2*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+2*weighted_std_rho_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-3*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+3*weighted_std_rho_HA, alpha=0.2, color='orange', label='3 $\sigma$')

    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HE = np.average(param_HE[:, 1], weights=1/err_HE[:, 1]**2)
    weighted_std_rho_HE = np.sqrt(np.average((param_HE[:, 1] - weighted_avg_rho_HE)**2, weights=1/err_HE[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax4.errorbar(date_in_transit_HE, param_HE[:, 1], yerr = err_HE[:, 1], fmt='.', label='Retrieved ')
    ax4.plot(date_in_transit_HE, sinusoid_rho_HE(date_in_transit_HE, param_rho_HE[0], param_rho_HE[1], param_rho_HE[2]), 'm', label='Model')
    ax4.set_xlabel('Time (MJD)')
    ax4.set_ylabel('Density ($m^{-3}$)')
    ax4.set_title('Best-fit column density for HE')
    ax4.axhline(density, color='k', label='Expected')
    ax4.axhline(weighted_avg_rho_HE, color='r', label='Average')
    ax4.axhline(-param_master_HE[1], color='b', label='Master')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-weighted_std_rho_HE, y2 = weighted_avg_rho_HE+weighted_std_rho_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-2*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+2*weighted_std_rho_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-3*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+3*weighted_std_rho_HE, alpha=0.2, color='orange', label='3 $\sigma$')

    ax1.sharey(ax2)
    ax3.sharey(ax4)

    fig.tight_layout()
    if save:
        plt.savefig(location+'/Plot_Results/Sin_Retrieved_Params.pdf')
        
        
    fig, (ax1, ax2)

#If there is one mode of observation
else:
    
    fig , (ax1, ax2) = plt.subplots(1, 2, figsize=[14, 6])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T = np.average(base_param[:, 0], weights=1/base_err[:, 0]**2)
    weighted_std_T = np.sqrt(np.average((base_param[:, 0] - weighted_avg_T)**2, weights=1/base_err[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit, base_param[:, 0], yerr = base_err[:, 0], fmt='.', label='Retrieved')
    ax1.plot(date_in_transit, sinusoid_T(date_in_transit, base_param_T[0], base_param_T[1], base_param_T[2]), 'm', label='Model')
    ax1.set_xlabel('Time (MJD)')
    ax1.set_ylabel('Temperature (K)')
    ax1.set_title('Best-fit temperature')
    ax1.axhline(temperature, color='k', label='Expected')
    ax1.axhline(weighted_avg_T, color='r', label='Average')
    ax1.axhline(base_param_master[0], color='b', label='Master')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-weighted_std_T, y2 = weighted_avg_T+weighted_std_T, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-2*weighted_std_T, y2 = weighted_avg_T+2*weighted_std_T, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-3*weighted_std_T, y2 = weighted_avg_T+3*weighted_std_T, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(framealpha=.7)

    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho = np.average(base_param[:, 1], weights=1/base_err[:, 1]**2)
    weighted_std_rho = np.sqrt(np.average((base_param[:, 1] - weighted_avg_rho)**2, weights=1/base_err[:, 1]**2))
        
    #Plotting the retrieved densities.
    ax2.errorbar(date_in_transit, base_param[:, 1], yerr = base_err[:, 1], fmt='.', label='Retrieved')
    ax2.plot(date_in_transit, sinusoid_rho(date_in_transit, base_param_rho[0], base_param_rho[1], base_param_rho[2]), 'm', label='Model')
    ax2.set_xlabel('Time (MJD)')
    ax2.set_ylabel('Density ($m^{-3}$)')
    ax2.set_title('Best-fit column density')
    ax2.axhline(density, color='k', label='Expected')
    ax2.axhline(weighted_avg_rho, color='r', label='Average')
    ax2.axhline(-base_param_master[1], color='b', label='Master')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-weighted_std_rho, y2 = weighted_avg_rho+weighted_std_rho, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-2*weighted_std_rho, y2 = weighted_avg_rho+2*weighted_std_rho, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-3*weighted_std_rho, y2 = weighted_avg_rho+3*weighted_std_rho, alpha=0.2, color='orange', label='3 $\sigma$')

    fig.tight_layout()
    if save:
        plt.savefig(location+'/Plot_Results/Sin_Retrieved_Params.pdf')
           

# Log column density

## Fitting absorption spectra

In [ ]:
reload(AF)

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    #Re-defining the absorption function
    #We fix the stellar velocity because the Sun's velocity is well known.
    #We fit for the atmospheric He column density and the temperature of the atmosphere.
    def log_absorption_function_HA(x, temp, log_density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_HA[0].shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HA)

    def log_absorption_function_HE(x, temp, log_density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_HE[0].shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HE)

    #Creating arrays to store the best-fit parameters and the error on them
    log_param_HA = np.zeros((len(diff_HA), 3))
    log_err_HA = np.zeros((len(diff_HA), 3))

    log_param_HE = np.zeros((len(diff_HE), 3))
    log_err_HE = np.zeros((len(diff_HE), 3))

    log_ref_wave = rassine_total_lamda_HA[0]
    #Iterating over the time series to get a value of the retrieved parameters for each spectrum
    for i in range(len(diff_HA)):
        
        #Guess on the initial parameters
        guess = [16000, 14, np.median(diff_HA[i])]

        #Chi-squared minimization routine
        params, cov = curve_fit(log_absorption_function_HA, log_ref_wave, diff_HA[i], sigma = diff_HA_err[i], p0=guess)

        #Storing the best-fit parameters and errors
        log_param_HA[i] = params
        log_err_HA[i] = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        print(params)
        model = log_absorption_function_HA(log_ref_wave, params[0], params[1], params[2])

        plt.figure(figsize=[10, 6])
        plt.errorbar(log_ref_wave, 100*(diff_HA[i]-offset), yerr=100*diff_HA_err[i], fmt='b.', label='Simulated data')
        plt.plot(log_ref_wave, 100*(log_absorption_function_HA(log_ref_wave, temperature, np.log10(density), params[2])-offset), color='orange', label='Expected result')
        plt.plot(log_ref_wave, 100*(model-offset), 'r', label='Best-fit')
        plt.xlim([10825, 10840])
        plt.xlabel('Wavelength ($\AA$)')
        plt.ylabel('Absorption (%)')
        plt.ylim(-2, 7)
        plt.legend()
        plt.show()
    
    log_ref_wave = rassine_total_lamda_HE[0]
    for i in range(len(diff_HE)):
        
        #Guess on the initial parameters
        guess = [16000, 14, np.median(diff_HE[i])]
        
        bnds = ((-np.inf, -np.inf, -np.inf), (20000, np.inf, np.inf))

        #Chi-squared minimization routine
        params, cov = curve_fit(log_absorption_function_HE, log_ref_wave, diff_HE[i], sigma=diff_HE_err[i], p0=guess, bounds=bnds)

        #Storing the best-fit parameters and errors
        log_param_HE[i] = params
        log_err_HE[i] = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        print(params)
        model = log_absorption_function_HE(log_ref_wave, params[0], params[1], params[2])

        plt.figure(figsize=[10, 6])
        plt.errorbar(log_ref_wave, 100*(diff_HE[i]-offset), yerr = 100*diff_HE_err[i], fmt='b.', label='Simulated data')
        plt.plot(log_ref_wave, 100*(log_absorption_function_HE(log_ref_wave, temperature, np.log10(density), params[2])-offset), color='orange', label='Expected result')
        plt.plot(log_ref_wave, 100*(model-offset), 'k', label='Best-fit')
        plt.xlim([10825, 10840])
        plt.xlabel('Wavelength ($\AA$)')
        plt.ylabel('Absorption (%)')
        plt.ylim(-2, 7)
        plt.legend()
        plt.show()

#If there is one mode of observation
else:
    #Re-defining the absorption function
    #We fix the stellar velocity because the Sun's velocity is well known.
    #We fit for the atmospheric He column density and the temperature of the atmosphere.
    def log_absorption_function(x, temp, log_density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff[0].shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power)

    #Creating arrays to store the best-fit parameters and the error on them
    log_param = np.zeros((len(diff), 3))
    log_err = np.zeros((len(diff), 3))
    
    log_ref_wave = rassine_total_lamda[0]
    #Iterating over the time series to get a value of the retrieved parameters for each spectrum
    for i in range(len(diff)):
        
        guess = [16000, 15, np.median(diff[i])]

        bnds = ((1000, 14, 0.9), (20000, 16, 1.2))
        
        #Chi-squared minimization routine
        params, cov = curve_fit(log_absorption_function, log_ref_wave, diff[i], sigma=diff_err[i], p0=guess, bounds=bnds)

        #Storing the best-fit parameters and errors
        log_param[i] = params
        print(params)
        log_err[i] = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        model = log_absorption_function(log_ref_wave, params[0], params[1], params[2])

        fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3, 1]}, figsize=[10, 6])
        ax1.axvline(10832.057472, color='lightgrey', label='He triplet location')
        ax1.axvline(10833.216751, color='lightgrey')
        ax1.axvline(10833.306444, color='lightgrey')
        ax1.errorbar(log_ref_wave, 100*(diff[i]-offset), yerr=100*diff_err[i], fmt='b.', label='Simulated data')
        ax1.plot(log_ref_wave, 100*(log_absorption_function(log_ref_wave, temperature, np.log10(density), params[2])-offset-0.02), color='orange', label='Expected result')
        ax1.plot(log_ref_wave, 100*(model-offset), 'r', label='Best-fit')
        ax1.set_xlim([10825, 10840])
        ax1.set_xlabel('Wavelength ($\AA$)')
        ax1.set_ylabel('Absorption (%)')
        ax2.errorbar(log_ref_wave, 100*(model-offset) - 100*(diff[i]-offset), yerr=100*diff_err[i], fmt='r.')
        plt.subplots_adjust(hspace=0)
        ax2.set_ylabel('Residuals (%)')
        ax2.set_ylim(-1, 1.2)
        ax1.legend()
        if i==0:
            plt.savefig('/Users/samsonmercier/Downloads/Mock_abs_spctr.pdf')
        plt.show()
        
        
'''if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    err_HA[:, 1] = err_HA[:, 1] * 10**param_HA[:, 1] * np.log(10)
    param_HA[:, 1] = 10**param_HA[:, 1]
    
    err_HE[:, 1] = err_HE[:, 1] * 10**param_HE[:, 1] * np.log(10)
    param_HE[:, 1] = 10**param_HE[:, 1]
    
else:
    err[:, 1] = err[:, 1] * 10**param[:, 1] * np.log(10)
    param[:, 1] = 10**param[:, 1]
    '''

## Double checking with master absorption spectrum

In [ ]:

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
        
    log_master_ref_wave = rassine_master_lamda_HA
    
    diff_master_HA = offset+(master_outtransit_HA - master_planet_intransit_HA)/master_outtransit_HA

    diff_master_err_HA = np.sqrt((master_err_intransit_HA/master_outtransit_HA)**2 + ((master_planet_intransit_HA * master_err_outtransit_HA)/(master_outtransit_HA**2))**2)

    def log_master_absorption_function_HA(x, temp, log_density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_master_HA.shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HA)

    guess = [16000, 14, np.median(diff_master_HA)]

    #Chi-squared minimization routine
    if using_Rassine:
        params, cov = curve_fit(log_master_absorption_function_HA, log_master_ref_wave, diff_master_HA, p0=guess)
    else:
        params, cov = curve_fit(log_master_absorption_function_HA, log_master_ref_wave, diff_master_HA, sigma=diff_master_err_HA, p0=guess)

    #Storing the best-fit parameters and errors
    log_param_master_HA = params
    log_err_master_HA = np.sqrt(np.diag(cov))

    #Plotting the in-transit planetary spectra and their corresponding model
    print(params)
    plt.figure(figsize=[10, 6])
    model = log_master_absorption_function_HA(log_master_ref_wave, params[0], params[1], params[2])
    
    plt.errorbar(log_master_ref_wave, 100*(diff_master_HA-offset), 100*diff_master_err_HA, fmt='b.', label='Simulated data')
    plt.plot(log_master_ref_wave, 100*(log_master_absorption_function_HA(log_master_ref_wave, temperature, np.log10(density), params[2])-offset), color='orange', label='Expected result')
    plt.plot(log_master_ref_wave, 100*(model-offset), 'r', label='Best-fit')
    plt.xlim([10825, 10840])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Absorption (%)')
    plt.legend()
    plt.show()

    log_master_ref_wave = rassine_master_lamda_HE
    
    diff_master_HE = offset+(master_outtransit_HE - master_planet_intransit_HE)/master_outtransit_HE

    diff_master_err_HE = np.sqrt((master_err_intransit_HE/master_outtransit_HE)**2 + ((master_planet_intransit_HE * master_err_outtransit_HE)/(master_outtransit_HE**2))**2)

    def log_master_absorption_function_HE(x, temp, log_density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_master_HE.shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HE)

    guess = [16000, 14, np.median(diff_master_HE)]

    #Chi-squared minimization routine
    if using_Rassine:
        params, cov = curve_fit(log_master_absorption_function_HE, log_master_ref_wave, diff_master_HE, p0=guess)
    else:
        params, cov = curve_fit(log_master_absorption_function_HE, log_master_ref_wave, diff_master_HE, sigma=diff_master_err_HE, p0=guess)
    #Storing the best-fit parameters and errors
    log_param_master_HE = params
    log_err_master_HE = np.sqrt(np.diag(cov))

    #Plotting the in-transit planetary spectra and their corresponding model
    print(params)
    plt.figure(figsize=[10, 6])
    model = log_master_absorption_function_HE(log_master_ref_wave, params[0], params[1], params[2])
    
    plt.errorbar(log_master_ref_wave, 100*(diff_master_HE-offset), 100*diff_master_err_HE, fmt='b.', label='Simulated data')
    plt.plot(log_master_ref_wave, 100*(log_master_absorption_function_HE(log_master_ref_wave, temperature, np.log10(density), params[2])-offset), color='orange', label='Expected result')
    plt.plot(log_master_ref_wave, 100*(model-offset), 'k', label='Best-fit')
    plt.xlim([10825, 10840])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Absorption (%)')
    plt.legend()
    plt.show()
    
    
else:
    log_master_ref_wav = rassine_master_lamda
    
    diff_master = offset+(master_outtransit - master_planet_intransit)/master_outtransit

    diff_master_err = np.sqrt((master_err_intransit/master_outtransit)**2 + ((master_planet_intransit * master_err_outtransit)/(master_outtransit**2))**2)

    def log_master_absorption_function(x, temp, log_density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_master.shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power)

    guess = [16000, 14, np.median(diff_master)]

    #Chi-squared minimization routine
    if using_Rassine:
        params, cov = curve_fit(log_master_absorption_function, log_master_ref_wav, diff_master, p0=guess)
    else:
        params, cov = curve_fit(log_master_absorption_function, log_master_ref_wav, diff_master, sigma=diff_master_err, p0=guess)

    #Storing the best-fit parameters and errors
    log_param_master = params
    log_err_master = np.sqrt(np.diag(cov))

    #Plotting the in-transit planetary spectra and their corresponding model
    print(params)
    model = log_master_absorption_function(log_master_ref_wav, params[0], params[1], params[2])

    plt.figure(figsize=[10, 6])
    plt.errorbar(log_master_ref_wav, 100*(diff_master-offset), 100*diff_master_err, fmt='b.', label='Simulated data')
    plt.plot(log_master_ref_wav, 100*(log_master_absorption_function(log_master_ref_wav, temperature, np.log10(density), params[2])-offset), color='orange', label='Expected result')
    plt.plot(log_master_ref_wav, 100*(model-offset), 'r', label='Best-fit')
    plt.xlim([10825, 10840])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Absorption (%)')
    plt.legend()
    plt.show()
    
'''if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    err_master_HA[1] = err_master_HA[1] * 10**param_master_HA[1] * np.log(10)
    param_master_HA[1] = 10**param_master_HA[1]
    
    err_master_HE[1] = err_master_HE[1] * 10**param_master_HE[1] * np.log(10)
    param_master_HE[1] = 10**param_master_HE[1]
      
else:
    err_master[1] = err_master[1] * 10**param_master[1] * np.log(10)
    param_master[1] = 10**param_master[1]
'''

## Plotting best-fit parameters

In [ ]:
#Retrieving depth measurements
# reference_He_wavelength = [10833.306444]

bestfit_depth = np.zeros(log_param.shape[0])

# eval_model = spec_mod.spectrum_he(log_ref_wave, log_param[5,2]+np.zeros(log_ref_wave.shape), log_param[5, 0], -10**log_param[5,1], vsini, 'VACUUM', RV_offset_He, R_pow = R_power)

# eval_model_one = spec_mod.spectrum_he(reference_He_wavelength[0], log_param[5,2], log_param[5, 0], -10**log_param[5,1], vsini, 'VACUUM', RV_offset_He, R_pow = R_power)

# eval_model_two = spec_mod.spectrum_he(log_ref_wave[np.where(eval_model==np.max(eval_model))[0][0]], log_param[5,2], log_param[5, 0], -10**log_param[5,1], vsini, 'VACUUM', RV_offset_He, R_pow = R_power)

# plt.plot(log_ref_wave, eval_model)
# plt.axvline(reference_He_wavelength)
# # plt.axvline(ref_wave[np.where(eval_model==np.max(eval_model))[0][0]])
# plt.xlim([10830, 10840])

# print(eval_model_one - log_param[5,2])
# print(eval_model_two - log_param[5,2])


for i in range(len(log_param)):
    location_peak = log_ref_wave[np.where(eval_model==np.max(eval_model))[0][0]]
    bestfit_depth[i] = spec_mod.spectrum_he(location_peak, log_param[i, 2], log_param[i, 0], -10**log_param[i, 1], vsini, 'VACUUM', RV_offset_He, R_pow = R_power) - log_param[i, 2]

In [ ]:
reload(AF)

amu=1.660531e-27    # kg
m_He = 4.002602 * amu

#Defining a variable to save or not the results.
save = True

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=[18, 12])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HA = np.average(log_param_HA[:, 0], weights=1/log_err_HA[:, 0]**2)
    weighted_std_T_HA = np.sqrt(np.average((log_param_HA[:, 0] - weighted_avg_T_HA)**2, weights=1/log_err_HA[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit_HA, log_param_HA[:, 0], yerr = log_err_HA[:, 0], fmt='.', label='Retrieved')
    ax1.set_xlabel('Time (MJD)')
    ax1.set_ylabel('Temperature (K)')
    ax1.set_title('(Logfit) Best-fit temperature for HA')
    ax1.axhline(temperature, color='k', label='Expected')
    ax1.axhline(weighted_avg_T_HA, color='r', label='Average')
    ax1.axhline(log_param_master_HA[0], color='b', label='Master')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-weighted_std_T_HA, y2 = weighted_avg_T_HA+weighted_std_T_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-2*weighted_std_T_HA, y2 = weighted_avg_T_HA+2*weighted_std_T_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-3*weighted_std_T_HA, y2 = weighted_avg_T_HA+3*weighted_std_T_HA, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(framealpha=0.8)
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HE = np.average(log_param_HE[:, 0], weights=1/log_err_HE[:, 0]**2)
    weighted_std_T_HE = np.sqrt(np.average((log_param_HE[:, 0] - weighted_avg_T_HE)**2, weights=1/log_err_HE[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax2.errorbar(date_in_transit_HE, log_param_HE[:, 0], yerr = log_err_HE[:, 0], fmt='.', label='Retrieved')
    ax2.set_xlabel('Time (MJD)')
    ax2.set_ylabel('Temperature (K)')
    ax2.set_title('(Logfit) Best-fit temperature for HE')
    ax2.axhline(temperature, color='k', label='Expected')
    ax2.axhline(weighted_avg_T_HE, color='r', label='Average')
    ax2.axhline(log_param_master_HE[0], color='b', label='Master')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-weighted_std_T_HE, y2 = weighted_avg_T_HE+weighted_std_T_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-2*weighted_std_T_HE, y2 = weighted_avg_T_HE+2*weighted_std_T_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-3*weighted_std_T_HE, y2 = weighted_avg_T_HE+3*weighted_std_T_HE, alpha=0.2, color='orange', label='3 $\sigma$')
    
    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HA = np.average(log_param_HA[:, 1], weights=1/log_err_HA[:, 1]**2)
    weighted_std_rho_HA = np.sqrt(np.average((log_param_HA[:, 1] - weighted_avg_rho_HA)**2, weights=1/log_err_HA[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax3.errorbar(date_in_transit_HA, log_param_HA[:, 1], yerr = log_err_HA[:, 1], fmt='.', label='Retrieved')
    ax3.set_xlabel('Time (MJD)')
    ax3.set_ylabel('Density ($m^{-3}$)')
    ax3.set_title('(Logfit) Best-fit column density for HA')
    ax3.axhline(np.log10(density), color='k', label='Expected')
    ax3.axhline(weighted_avg_rho_HA, color='r', label='Average')
    ax3.axhline(log_param_master_HA[1], color='b', label='Master')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-weighted_std_rho_HA, y2 = weighted_avg_rho_HA+weighted_std_rho_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-2*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+2*weighted_std_rho_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-3*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+3*weighted_std_rho_HA, alpha=0.2, color='orange', label='3 $\sigma$')

    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HE = np.average(log_param_HE[:, 1], weights=1/log_err_HE[:, 1]**2)
    weighted_std_rho_HE = np.sqrt(np.average((log_param_HE[:, 1] - weighted_avg_rho_HE)**2, weights=1/log_err_HE[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax4.errorbar(date_in_transit_HE, log_param_HE[:, 1], yerr = log_err_HE[:, 1], fmt='.', label='Retrieved ')
    ax4.set_xlabel('Time (MJD)')
    ax4.set_ylabel('Density ($m^{-3}$)')
    ax4.set_title('(Logfit) Best-fit column density for HE')
    ax4.axhline(np.log10(density), color='k', label='Expected')
    ax4.axhline(weighted_avg_rho_HE, color='r', label='Average')
    ax4.axhline(log_param_master_HE[1], color='b', label='Master')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-weighted_std_rho_HE, y2 = weighted_avg_rho_HE+weighted_std_rho_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-2*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+2*weighted_std_rho_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-3*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+3*weighted_std_rho_HE, alpha=0.2, color='orange', label='3 $\sigma$')

    ax1.sharey(ax2)
    ax3.sharey(ax4)

    fig.tight_layout()
    if save:
        plt.savefig(save_location+'Log_n_Retrieved_Params.pdf')
    
    print('HA - Expected T is: '+str(temperature)+' and retrieved T is: %.2f'%weighted_avg_T_HA,' and difference is: %.2f'%np.abs((temperature - weighted_avg_T_HA)/weighted_std_T_HA), ' sigma')
    print('HA - Expected density is: '+str(density/1e14)+' and retrieved density is: %.2f'%(10**weighted_avg_rho_HA / 1e14),' and difference is: %.2f'%np.abs((np.log10(density) - weighted_avg_rho_HA)/weighted_std_rho_HA), ' sigma')
    print('HE - Expected T is: '+str(temperature)+' and retrieved T is: %.2f'%weighted_avg_T_HE,' and difference is: %.2f'%np.abs((temperature - weighted_avg_T_HE)/weighted_std_T_HE), ' sigma')
    print('HE - Expected density is: '+str(density/1e14)+' and retrieved density is: %.2f'%(10**weighted_avg_rho_HE / 1e14),' and difference is: %.2f'%np.abs((np.log10(density) - weighted_avg_rho_HE)/weighted_std_rho_HE), ' sigma')
    
#If there is one mode of observation
else:
    
    fig , (ax1, ax2) = plt.subplots(1, 2, figsize=[14, 8])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T = np.average(log_param[:, 0], weights=1/log_err[:, 0]**2)
    weighted_std_T = np.sqrt(np.average((log_param[:, 0] - weighted_avg_T)**2, weights=1/log_err[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit-59970, log_param[:, 0], yerr = log_err[:, 0], fmt='.', label='Retrieved')
    ax1.set_xlabel('Time (MJD-59970)', fontsize=14)
    ax1.set_ylabel('Temperature (K)', fontsize=14)
    #ax1.set_title('Atmospheric temperature', fontsize=14)
    ax1.axhline(temperature, color='k', label='Expected')
    ax1.axhline(weighted_avg_T, color='r', label='Average')
    ax1.axhline(log_param_master[0], color='b', label='Master')
    ax1.tick_params(axis='both', labelsize=14)
    ax1.fill_between(date_in_transit-59970, y1 = weighted_avg_T-weighted_std_T, y2 = weighted_avg_T+weighted_std_T, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit-59970, y1 = weighted_avg_T-2*weighted_std_T, y2 = weighted_avg_T+2*weighted_std_T, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit-59970, y1 = weighted_avg_T-3*weighted_std_T, y2 = weighted_avg_T+3*weighted_std_T, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(fontsize=14, framealpha=.7, ncol=2, loc='upper right')

    #Plotting the retrieved temperatures but in width 
    ax1twin = ax1.twinx()
    widths = AF.Temp_to_Width(m_He, log_param[:, 0], R_power)
    ax1twin.plot(date_in_transit-59970, widths/1000, '.', alpha=0)
    ax1twin.set_ylabel('Width (km/s)', fontsize=14)
    ax1twin.tick_params(axis='y', labelsize=14)
    
    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho = np.average(log_param[:, 1], weights=1/log_err[:, 1]**2)
    weighted_std_rho = np.sqrt(np.average((log_param[:, 1] - weighted_avg_rho)**2, weights=1/log_err[:, 1]**2))
        
    #Plotting the retrieved densities.
    ax2.errorbar(date_in_transit-59970, log_param[:, 1], yerr = log_err[:, 1], fmt='.', label='Retrieved')
    ax2.set_xlabel('Time (MJD-59970)', fontsize=14)
    ax2.set_ylabel('Log$_{10}$ Column Density ($cm^{-2}$)', fontsize=14)
    #ax2.set_title('Helium column density', fontsize=14)
    ax2.axhline(np.log10(density), color='k', label='Expected')
    ax2.axhline(weighted_avg_rho, color='r', label='Average')
    ax2.axhline(log_param_master[1], color='b', label='Master')
    ax2.tick_params(axis='both', labelsize=14)
    ax2.fill_between(date_in_transit-59970, y1 = weighted_avg_rho-weighted_std_rho, y2 = weighted_avg_rho+weighted_std_rho, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit-59970, y1 = weighted_avg_rho-2*weighted_std_rho, y2 = weighted_avg_rho+2*weighted_std_rho, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit-59970, y1 = weighted_avg_rho-3*weighted_std_rho, y2 = weighted_avg_rho+3*weighted_std_rho, alpha=0.2, color='orange', label='3 $\sigma$')

    #Plotting the retrieved densities but in depth
    ax2twin = ax2.twinx()
    ax2twin.plot(date_in_transit-59970, 100*bestfit_depth, '.', alpha=0)
    ax2twin.set_ylabel('Depth (%)', fontsize=14)
    ax2twin.tick_params(axis='y', labelsize=14)
    
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Log_n_Retrieved_Params.pdf')
#         plt.savefig(save_location+'Log_n_Retrieved_Params.pdf')
        
    print('Expected T is: '+str(temperature)+' and retrieved T is: %.2f'%weighted_avg_T,' and difference is: %.2f'%np.abs((temperature - weighted_avg_T)/weighted_std_T), ' sigma')
    print('Expected density is: '+str(density/1e14)+' and retrieved density is: %.2f'%(10**weighted_avg_rho / 1e14),' and difference is: %.2f'%np.abs((np.log10(density) - weighted_avg_rho)/weighted_std_rho), ' sigma')


## Periodogram

In [ ]:
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    LS_T_HA = LombScargle(date_in_transit_HA, log_param_HA[:, 0], log_err_HA[:, 0]).autopower(nyquist_factor=1)
    LS_T_HE = LombScargle(date_in_transit_HE, log_param_HE[:, 0], log_err_HE[:, 0]).autopower(nyquist_factor=1)
    
    LS_rho_HA = LombScargle(date_in_transit_HA, log_param_HA[:, 1], log_err_HA[:, 1]).autopower(nyquist_factor=1)
    LS_rho_HE = LombScargle(date_in_transit_HE, log_param_HE[:, 1], log_err_HE[:, 1]).autopower(nyquist_factor=1)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=[10, 4])
    ax1.loglog(LS_T_HA[0], LS_T_HA[1])
    ax1.set_xlabel('Frequency (1/d)')
    ax1.set_ylabel('Power')
    ax1.set_title('Temperature Periodogram for HA')
    ax1.axvline(LS_T_HA[0][LS_T_HA[1]==np.max(LS_T_HA[1])])
    
    ax2.loglog(LS_T_HE[0], LS_T_HE[1])
    ax2.set_xlabel('Frequency (1/d)')
    ax2.set_title('Temperature Periodogram for HE')
    ax2.axvline(LS_T_HE[0][LS_T_HE[1]==np.max(LS_T_HE[1])])
    print(LS_T_HA[0][LS_T_HA[1]==np.max(LS_T_HA[1])], LS_T_HE[0][LS_T_HE[1]==np.max(LS_T_HE[1])])
    
    fig.tight_layout()
    plt.show()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=[10, 4])
    ax1.loglog(LS_rho_HA[0], LS_rho_HA[1])
    ax1.set_xlabel('Frequency (1/d)')
    ax1.set_ylabel('Power')
    ax1.set_title('Column Density Periodogram for HA')
    ax1.axvline(LS_rho_HA[0][LS_rho_HA[1]==np.max(LS_rho_HA[1])])
    
    ax2.loglog(LS_rho_HE[0], LS_rho_HE[1])
    ax2.set_xlabel('Frequency (1/d)')
    ax2.set_title('Column Density Periodogram for HE')
    ax2.axvline(LS_rho_HE[0][LS_rho_HE[1]==np.max(LS_rho_HE[1])])
    print(LS_rho_HA[0][LS_rho_HA[1]==np.max(LS_rho_HA[1])], LS_rho_HE[0][LS_rho_HE[1]==np.max(LS_rho_HE[1])])

    fig.tight_layout()
    plt.show()
    
    
else:

    LS_T = LombScargle(date_in_transit, log_param[:, 0], log_err[:, 0]).autopower(nyquist_factor=1)
    
    LS_rho = LombScargle(date_in_transit, log_param[:, 1], log_err[:, 1]).autopower(nyquist_factor=1)
    
    plt.loglog(LS_T[0], LS_T[1])
    plt.xlabel('Frequency (1/d)')
    plt.ylabel('Power')
    plt.title('Temperature Periodogram for HA')
    plt.axvline(LS_T[0][LS_T[1]==np.max(LS_T[1])])
    print(LS_T[0][LS_T[1]==np.max(LS_T[1])])
    
    plt.show()
    
    plt.loglog(LS_rho[0], LS_rho[1])
    plt.xlabel('Frequency (1/d)')
    plt.ylabel('Power')
    plt.title('Column Density Periodogram for HA')
    plt.axvline(LS_rho[0][LS_rho[1]==np.max(LS_rho[1])])
    print(LS_rho[0][LS_rho[1]==np.max(LS_rho[1])])

    plt.show()


## Check if there's a sinusoidal trend

In [ ]:
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    ## Double check that omega_rho and omega_T get overwritten when switching modes
    omega_rho_HA = LS_rho_HA[0][LS_rho_HA[1]==np.max(LS_rho_HA[1])][0]
    omega_T_HA = LS_T_HA[0][LS_T_HA[1]==np.max(LS_T_HA[1])][0]

    def sinusoid_T_HA(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_T_HA) + B*np.sin(2*np.pi*t*omega_T_HA)

    def sinusoid_rho_HA(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_rho_HA) + B*np.sin(2*np.pi*t*omega_rho_HA)

    log_param_T_HA, log_cov_T_HA =  curve_fit(sinusoid_T_HA, date_in_transit_HA, log_param_HA[:, 0], sigma = log_err_HA[:, 0])
    log_param_rho_HA, log_cov_rho_HA =  curve_fit(sinusoid_rho_HA, date_in_transit_HA, log_param_HA[:, 1], sigma = log_err_HA[:, 1], p0=[5e13, 5e13, 4.6e14])

    omega_rho_HE = LS_rho_HE[0][LS_rho_HE[1]==np.max(LS_rho_HE[1])][0]
    omega_T_HE = LS_T_HE[0][LS_T_HE[1]==np.max(LS_T_HE[1])][0]

    def sinusoid_T_HE(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_T_HE) + B*np.sin(2*np.pi*t*omega_T_HE)

    def sinusoid_rho_HE(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_rho_HE) + B*np.sin(2*np.pi*t*omega_rho_HE)

    log_param_T_HE, log_cov_T_HE =  curve_fit(sinusoid_T_HE, date_in_transit_HE, log_param_HE[:, 0], sigma = log_err_HE[:, 0])
    log_param_rho_HE, log_cov_rho_HE =  curve_fit(sinusoid_rho_HE, date_in_transit_HE, log_param_HE[:, 1], sigma = log_err_HE[:, 1], p0=[5e13, 5e13, 4.6e14])

else:
    
    omega_rho = LS_rho[0][LS_rho[1]==np.max(LS_rho[1])]
    omega_T = LS_T[0][LS_T[1]==np.max(LS_T[1])]
    
    def sinusoid_T(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_T) + B*np.sin(2*np.pi*t*omega_T)

    def sinusoid_rho(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_rho) + B*np.sin(2*np.pi*t*omega_rho)


    log_param_T, log_cov_T =  curve_fit(sinusoid_T, date_in_transit, log_param[:, 0], sigma = log_err[:, 0])
    log_param_rho, log_cov_rho =  curve_fit(sinusoid_rho, date_in_transit, log_param[:, 1], sigma = log_err[:, 1], p0=[5e13, 5e13, 4.6e14])

    

In [ ]:
reload(AF)

#Defining a variable to save or not the results.
save = True

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=[18, 12])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HA = np.average(log_param_HA[:, 0], weights=1/log_err_HA[:, 0]**2)
    weighted_std_T_HA = np.sqrt(np.average((log_param_HA[:, 0] - weighted_avg_T_HA)**2, weights=1/log_err_HA[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit_HA, log_param_HA[:, 0], yerr = log_err_HA[:, 0], fmt='.', label='Retrieved')
    ax1.plot(date_in_transit_HA, sinusoid_T_HA(date_in_transit_HA, log_param_T_HA[0], log_param_T_HA[1], log_param_T_HA[2]), 'm', label='Model')
    ax1.set_xlabel('Time (MJD)')
    ax1.set_ylabel('Temperature (K)')
    ax1.set_title('(Logfit) Best-fit temperature for HA')
    ax1.axhline(temperature, color='k', label='Expected')
    ax1.axhline(weighted_avg_T_HA, color='r', label='Average')
    ax1.axhline(log_param_master_HA[0], color='b', label='Master')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-weighted_std_T_HA, y2 = weighted_avg_T_HA+weighted_std_T_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-2*weighted_std_T_HA, y2 = weighted_avg_T_HA+2*weighted_std_T_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-3*weighted_std_T_HA, y2 = weighted_avg_T_HA+3*weighted_std_T_HA, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(framealpha=0.8)
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HE = np.average(log_param_HE[:, 0], weights=1/log_err_HE[:, 0]**2)
    weighted_std_T_HE = np.sqrt(np.average((log_param_HE[:, 0] - weighted_avg_T_HE)**2, weights=1/log_err_HE[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax2.errorbar(date_in_transit_HE, log_param_HE[:, 0], yerr = log_err_HE[:, 0], fmt='.', label='Retrieved')
    ax2.plot(date_in_transit_HE, sinusoid_T_HE(date_in_transit_HE, log_param_T_HE[0], log_param_T_HE[1], log_param_T_HE[2]), 'm', label='Model')
    ax2.set_xlabel('Time (MJD)')
    ax2.set_ylabel('Temperature (K)')
    ax2.set_title('(Logfit) Best-fit temperature for HE')
    ax2.axhline(temperature, color='k', label='Expected')
    ax2.axhline(weighted_avg_T_HE, color='r', label='Average')
    ax2.axhline(log_param_master_HE[0], color='b', label='Master')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-weighted_std_T_HE, y2 = weighted_avg_T_HE+weighted_std_T_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-2*weighted_std_T_HE, y2 = weighted_avg_T_HE+2*weighted_std_T_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-3*weighted_std_T_HE, y2 = weighted_avg_T_HE+3*weighted_std_T_HE, alpha=0.2, color='orange', label='3 $\sigma$')
    
    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HA = np.average(log_param_HA[:, 1], weights=1/log_err_HA[:, 1]**2)
    weighted_std_rho_HA = np.sqrt(np.average((log_param_HA[:, 1] - weighted_avg_rho_HA)**2, weights=1/log_err_HA[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax3.errorbar(date_in_transit_HA, log_param_HA[:, 1], yerr = log_err_HA[:, 1], fmt='.', label='Retrieved')
    ax3.plot(date_in_transit_HA, sinusoid_rho_HA(date_in_transit_HA, log_param_rho_HA[0], log_param_rho_HA[1], log_param_rho_HA[2]), 'm', label='Model')
    ax3.set_xlabel('Time (MJD)')
    ax3.set_ylabel('Density ($m^{-3}$)')
    ax3.set_title('(Logfit) Best-fit column density for HA')
    ax3.axhline(np.log10(density), color='k', label='Expected')
    ax3.axhline(weighted_avg_rho_HA, color='r', label='Average')
    ax3.axhline(log_param_master_HA[1], color='b', label='Master')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-weighted_std_rho_HA, y2 = weighted_avg_rho_HA+weighted_std_rho_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-2*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+2*weighted_std_rho_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-3*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+3*weighted_std_rho_HA, alpha=0.2, color='orange', label='3 $\sigma$')

    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HE = np.average(log_param_HE[:, 1], weights=1/log_err_HE[:, 1]**2)
    weighted_std_rho_HE = np.sqrt(np.average((log_param_HE[:, 1] - weighted_avg_rho_HE)**2, weights=1/log_err_HE[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax4.errorbar(date_in_transit_HE, log_param_HE[:, 1], yerr = log_err_HE[:, 1], fmt='.', label='Retrieved ')
    ax4.plot(date_in_transit_HE, sinusoid_rho_HE(date_in_transit_HE, log_param_rho_HE[0], log_param_rho_HE[1], log_param_rho_HE[2]), 'm', label='Model')
    ax4.set_xlabel('Time (MJD)')
    ax4.set_ylabel('Density ($m^{-3}$)')
    ax4.set_title('(Logfit) Best-fit column density for HE')
    ax4.axhline(np.log10(density), color='k', label='Expected')
    ax4.axhline(weighted_avg_rho_HE, color='r', label='Average')
    ax4.axhline(log_param_master_HE[1], color='b', label='Master')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-weighted_std_rho_HE, y2 = weighted_avg_rho_HE+weighted_std_rho_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-2*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+2*weighted_std_rho_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-3*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+3*weighted_std_rho_HE, alpha=0.2, color='orange', label='3 $\sigma$')

    ax1.sharey(ax2)
    ax3.sharey(ax4)

    fig.tight_layout()
    if save:
        plt.savefig(location+'/Plot_Results/Log_Sin_Retrieved_Params.pdf')
        
        
    fig, (ax1, ax2)

#If there is one mode of observation
else:
    
    fig , (ax1, ax2) = plt.subplots(1, 2, figsize=[14, 6])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T = np.average(log_param[:, 0], weights=1/log_err[:, 0]**2)
    weighted_std_T = np.sqrt(np.average((log_param[:, 0] - weighted_avg_T)**2, weights=1/log_err[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit, log_param[:, 0], yerr = log_err[:, 0], fmt='.', label='Retrieved')
    ax1.plot(date_in_transit, sinusoid_T(date_in_transit, log_param_T[0], log_param_T[1], log_param_T[2]), 'm', label='Model')
    ax1.set_xlabel('Time (MJD)')
    ax1.set_ylabel('Temperature (K)')
    ax1.set_title('(Logfit) Best-fit temperature')
    ax1.axhline(temperature, color='k', label='Expected')
    ax1.axhline(weighted_avg_T, color='r', label='Average')
    ax1.axhline(log_param_master[0], color='b', label='Master')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-weighted_std_T, y2 = weighted_avg_T+weighted_std_T, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-2*weighted_std_T, y2 = weighted_avg_T+2*weighted_std_T, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-3*weighted_std_T, y2 = weighted_avg_T+3*weighted_std_T, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(framealpha=.7)

    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho = np.average(log_param[:, 1], weights=1/log_err[:, 1]**2)
    weighted_std_rho = np.sqrt(np.average((log_param[:, 1] - weighted_avg_rho)**2, weights=1/log_err[:, 1]**2))
        
    #Plotting the retrieved densities.
    ax2.errorbar(date_in_transit, log_param[:, 1], yerr = log_err[:, 1], fmt='.', label='Retrieved')
    ax2.plot(date_in_transit, sinusoid_rho(date_in_transit, log_param_rho[0], log_param_rho[1], log_param_rho[2]), 'm', label='Model')
    ax2.set_xlabel('Time (MJD)')
    ax2.set_ylabel('Density ($m^{-3}$)')
    ax2.set_title('(Logfit) Best-fit column density')
    ax2.axhline(np.log10(density), color='k', label='Expected')
    ax2.axhline(weighted_avg_rho, color='r', label='Average')
    ax2.axhline(log_param_master[1], color='b', label='Master')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-weighted_std_rho, y2 = weighted_avg_rho+weighted_std_rho, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-2*weighted_std_rho, y2 = weighted_avg_rho+2*weighted_std_rho, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-3*weighted_std_rho, y2 = weighted_avg_rho+3*weighted_std_rho, alpha=0.2, color='orange', label='3 $\sigma$')

    fig.tight_layout()
    if save:
        plt.savefig(location+'/Plot_Results/Log_Sin_Retrieved_Params.pdf')

# Log column density + Log temperature

## Fitting absorption spectra

In [ ]:
reload(AF)

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    #Re-defining the absorption function
    #We fix the stellar velocity because the Sun's velocity is well known.
    #We fit for the atmospheric He column density and the temperature of the atmosphere.
    def loglog_absorption_function_HA(x, log_temp, log_density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_HA[0].shape), 10**log_temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HA)

    def loglog_absorption_function_HE(x, log_temp, log_density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_HE[0].shape), 10**log_temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HE)

    #Creating arrays to store the best-fit parameters and the error on them
    loglog_param_HA = np.zeros((len(diff_HA), 3))
    loglog_err_HA = np.zeros((len(diff_HA), 3))

    loglog_param_HE = np.zeros((len(diff_HE), 3))
    loglog_err_HE = np.zeros((len(diff_HE), 3))

    loglog_ref_wave = rassine_total_lamda_HA[0]
    #Iterating over the time series to get a value of the retrieved parameters for each spectrum
    for i in range(len(diff_HA)):
        
        #Guess on the initial parameters
        guess = [np.log10(16000), 14, np.median(diff_HA[i])]

        #Chi-squared minimization routine
        params, cov = curve_fit(loglog_absorption_function_HA, loglog_ref_wave, diff_HA[i], sigma = diff_HA_err[i], p0=guess)

        #Storing the best-fit parameters and errors
        loglog_param_HA[i] = params
        loglog_err_HA[i] = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        print(params)
        model = loglog_absorption_function_HA(loglog_ref_wave, params[0], params[1], params[2])

        plt.figure(figsize=[10, 6])
        plt.errorbar(loglog_ref_wave, 100*(diff_HA[i]-offset), yerr=100*diff_HA_err[i], fmt='b.', label='Simulated data')
        plt.plot(loglog_ref_wave, 100*(loglog_absorption_function_HA(loglog_ref_wave, np.log10(temperature), np.log10(density), params[2])-offset), color='orange', label='Expected result')
        plt.plot(loglog_ref_wave, 100*(model-offset), 'r', label='Best-fit')
        plt.xlim([10825, 10840])
        plt.xlabel('Wavelength ($\AA$)')
        plt.ylabel('Absorption (%)')
        plt.ylim(-2, 7)
        plt.legend()
        plt.show()
    
    loglog_ref_wave = rassine_total_lamda_HE[0]
    for i in range(len(diff_HE)):
        
        #Guess on the initial parameters
        guess = [np.log10(16000), 14, np.median(diff_HE[i])]
        
        bnds = ((-np.inf, -np.inf, -np.inf), (np.log10(20000), np.inf, np.inf))

        #Chi-squared minimization routine
        params, cov = curve_fit(loglog_absorption_function_HE, loglog_ref_wave, diff_HE[i], sigma=diff_HE_err[i], p0=guess, bounds=bnds)

        #Storing the best-fit parameters and errors
        loglog_param_HE[i] = params
        loglog_err_HE[i] = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        print(params)
        model = loglog_absorption_function_HE(loglog_ref_wave, params[0], params[1], params[2])

        plt.figure(figsize=[10, 6])
        plt.errorbar(loglog_ref_wave, 100*(diff_HE[i]-offset), yerr = 100*diff_HE_err[i], fmt='b.', label='Simulated data')
        plt.plot(loglog_ref_wave, 100*(loglog_absorption_function_HE(loglog_ref_wave, np.log10(temperature), np.log10(density), params[2])-offset), color='orange', label='Expected result')
        plt.plot(loglog_ref_wave, 100*(model-offset), 'k', label='Best-fit')
        plt.xlim([10825, 10840])
        plt.xlabel('Wavelength ($\AA$)')
        plt.ylabel('Absorption (%)')
        plt.ylim(-2, 7)
        plt.legend()
        plt.show()

#If there is one mode of observation
else:
    #Re-defining the absorption function
    #We fix the stellar velocity because the Sun's velocity is well known.
    #We fit for the atmospheric He column density and the temperature of the atmosphere.
    def loglog_absorption_function(x, log_temp, log_density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff[0].shape), 10**log_temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power)

    #Creating arrays to store the best-fit parameters and the error on them
    loglog_param = np.zeros((len(diff), 3))
    loglog_err = np.zeros((len(diff), 3))
    
    loglog_ref_wave = rassine_total_lamda[0]
    #Iterating over the time series to get a value of the retrieved parameters for each spectrum
    for i in range(len(diff)):
        
        guess = [np.log10(16000), 15, np.median(diff[i])]

        bnds = ((np.log10(1000), 14, 0.9), (np.log10(20000), 16, 1.2))
        
        #Chi-squared minimization routine
        params, cov = curve_fit(loglog_absorption_function, loglog_ref_wave, diff[i], sigma=diff_err[i], p0=guess, bounds=bnds)

        #Storing the best-fit parameters and errors
        loglog_param[i] = params
        print(params)
        loglog_err[i] = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        model = loglog_absorption_function(loglog_ref_wave, params[0], params[1], params[2])

        plt.figure(figsize=[10, 6])
        plt.errorbar(loglog_ref_wave, 100*(diff[i]-offset), 100*diff_err[i], fmt='b.', label='Simulated data')
        plt.plot(loglog_ref_wave, 100*(loglog_absorption_function(loglog_ref_wave, np.log10(temperature), np.log10(density), params[2])-offset), color='orange', label='Expected result')
        plt.plot(loglog_ref_wave, 100*(model-offset), 'r', label='Best-fit')
        plt.xlim([10825, 10840])
        plt.xlabel('Wavelength ($\AA$)')
        plt.ylabel('Absorption (%)')
        plt.legend()
        plt.show()
        
        
'''if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    err_HA[:, 1] = err_HA[:, 1] * 10**param_HA[:, 1] * np.log(10)
    param_HA[:, 1] = 10**param_HA[:, 1]
    
    err_HE[:, 1] = err_HE[:, 1] * 10**param_HE[:, 1] * np.log(10)
    param_HE[:, 1] = 10**param_HE[:, 1]
    
else:
    err[:, 1] = err[:, 1] * 10**param[:, 1] * np.log(10)
    param[:, 1] = 10**param[:, 1]
    '''

## Double checking with master spectrum

In [ ]:

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
        
    loglog_master_ref_wave = rassine_master_lamda_HA
    
    diff_master_HA = offset+(master_outtransit_HA - master_planet_intransit_HA)/master_outtransit_HA

    diff_master_err_HA = np.sqrt((master_err_intransit_HA/master_outtransit_HA)**2 + ((master_planet_intransit_HA * master_err_outtransit_HA)/(master_outtransit_HA**2))**2)

    def loglog_master_absorption_function_HA(x, log_temp, log_density, offset):
        return offset+spec_mod.spectrum_he(x, np.ones(diff_master_HA.shape), 10**log_temp, -10**log_density, vsini, 'VACUUM',RV_offset_He, R_pow = R_power_HA)

    guess = [np.log10(16000), 14, np.median(diff_master_HA)]

    #Chi-squared minimization routine
    if using_Rassine:
        params, cov = curve_fit(loglog_master_absorption_function_HA, loglog_master_ref_wave, diff_master_HA, p0=guess)
    else:
        params, cov = curve_fit(loglog_master_absorption_function_HA, loglog_master_ref_wave, diff_master_HA, sigma=diff_master_err_HA, p0=guess)

    #Storing the best-fit parameters and errors
    loglog_param_master_HA = params
    loglog_err_master_HA = np.sqrt(np.diag(cov))

    #Plotting the in-transit planetary spectra and their corresponding model
    print(params)
    plt.figure(figsize=[10, 6])
    model = loglog_master_absorption_function_HA(loglog_master_ref_wave, params[0], params[1], params[2])
    
    plt.errorbar(loglog_master_ref_wave, 100*(diff_master_HA-offset), 100*diff_master_err_HA, fmt='b.', label='Simulated data')
    plt.plot(loglog_master_ref_wave, 100*(loglog_master_absorption_function_HA(loglog_master_ref_wave, np.log10(temperature), np.log10(density), params[2])-offset), color='orange', label='Expected result')
    plt.plot(loglog_master_ref_wave, 100*(model-offset), 'r', label='Best-fit')
    plt.xlim([10825, 10840])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Absorption (%)')
    plt.legend()
    plt.show()

    loglog_master_ref_wave = rassine_master_lamda_HE
    
    diff_master_HE = offset+(master_outtransit_HE - master_planet_intransit_HE)/master_outtransit_HE

    diff_master_err_HE = np.sqrt((master_err_intransit_HE/master_outtransit_HE)**2 + ((master_planet_intransit_HE * master_err_outtransit_HE)/(master_outtransit_HE**2))**2)

    def loglog_master_absorption_function_HE(x, log_temp, log_density, offset):
        return offset+spec_mod.spectrum_he(x, np.ones(diff_master_HE.shape), 10**log_temp, -10**log_density, vsini, 'VACUUM',RV_offset_He, R_pow = R_power_HE)

    guess = [np.log10(16000), 14, np.median(diff_master_HE)]

    #Chi-squared minimization routine
    if using_Rassine:
        params, cov = curve_fit(loglog_master_absorption_function_HE, loglog_master_ref_wave, diff_master_HE, p0=guess)
    else:
        params, cov = curve_fit(loglog_master_absorption_function_HE, loglog_master_ref_wave, diff_master_HE, sigma=diff_master_err_HE, p0=guess)
    #Storing the best-fit parameters and errors
    loglog_param_master_HE = params
    loglog_err_master_HE = np.sqrt(np.diag(cov))

    #Plotting the in-transit planetary spectra and their corresponding model
    print(params)
    plt.figure(figsize=[10, 6])
    model = loglog_master_absorption_function_HE(loglog_master_ref_wave, params[0], params[1], params[2])
    plt.errorbar(loglog_master_ref_wave, 100*(diff_master_HE-offset), 100*diff_master_err_HE, fmt='b.', label='Simulated data')
    plt.plot(loglog_master_ref_wave, 100*(loglog_master_absorption_function_HE(loglog_master_ref_wave, np.log10(temperature), np.log10(density), params[2])-offset), color='orange', label='Expected result')
    plt.plot(loglog_master_ref_wave, 100*(model-offset), 'k', label='Best-fit')
    plt.xlim([10825, 10840])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Absorption (%)')
    plt.legend()
    plt.show()
    
    
else:
    loglog_master_ref_wav = rassine_master_lamda
    
    diff_master = offset+(master_outtransit - master_planet_intransit)/master_outtransit

    diff_master_err = np.sqrt((master_err_intransit/master_outtransit)**2 + ((master_planet_intransit * master_err_outtransit)/(master_outtransit**2))**2)

    def loglog_master_absorption_function(x, log_temp, log_density, offset):
        return spec_mod.spectrum_he(x, offset+np.zeros(diff_master.shape), 10**log_temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power)

    guess = [np.log10(16000), 14, np.median(diff_master)]

    #Chi-squared minimization routine
    if using_Rassine:
        params, cov = curve_fit(loglog_master_absorption_function, loglog_master_ref_wav, diff_master, p0=guess)
    else:
        params, cov = curve_fit(loglog_master_absorption_function, loglog_master_ref_wav, diff_master, sigma=diff_master_err, p0=guess)

    #Storing the best-fit parameters and errors
    loglog_param_master = params
    loglog_err_master = np.sqrt(np.diag(cov))

    #Plotting the in-transit planetary spectra and their corresponding model
    print(params)
    model = loglog_master_absorption_function(loglog_master_ref_wav, params[0], params[1], params[2])

    plt.figure(figsize=[10, 6])
    plt.errorbar(loglog_master_ref_wav, 100*(diff_master-offset), 100*diff_master_err, fmt='b.', label='Simulated data')
    plt.plot(loglog_master_ref_wav, 100*(loglog_master_absorption_function(loglog_master_ref_wav, np.log10(temperature), np.log10(density), params[2])-offset), color='orange', label='Expected result')
    plt.plot(loglog_master_ref_wav, 100*(model-offset), 'r', label='Best-fit')
    plt.xlim([10825, 10840])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Absorption (%)')
    plt.legend()
    plt.show()
    
'''if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    err_master_HA[1] = err_master_HA[1] * 10**param_master_HA[1] * np.log(10)
    param_master_HA[1] = 10**param_master_HA[1]
    
    err_master_HE[1] = err_master_HE[1] * 10**param_master_HE[1] * np.log(10)
    param_master_HE[1] = 10**param_master_HE[1]
      
else:
    err_master[1] = err_master[1] * 10**param_master[1] * np.log(10)
    param_master[1] = 10**param_master[1]
'''

## Plotting best-fit parameters

In [ ]:
reload(AF)

#Defining a variable to save or not the results.
save = True

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=[18, 12])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HA = np.average(loglog_param_HA[:, 0], weights=1/loglog_err_HA[:, 0]**2)
    weighted_std_T_HA = np.sqrt(np.average((loglog_param_HA[:, 0] - weighted_avg_T_HA)**2, weights=1/loglog_err_HA[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit_HA, loglog_param_HA[:, 0], yerr = loglog_err_HA[:, 0], fmt='.', label='Retrieved')
    ax1.set_xlabel('Time (MJD)')
    ax1.set_ylabel('Temperature (K)')
    ax1.set_title('(Logfit) Best-fit temperature for HA')
    ax1.axhline(np.log10(temperature), color='k', label='Expected')
    ax1.axhline(weighted_avg_T_HA, color='r', label='Average')
    ax1.axhline(loglog_param_master_HA[0], color='b', label='Master')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-weighted_std_T_HA, y2 = weighted_avg_T_HA+weighted_std_T_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-2*weighted_std_T_HA, y2 = weighted_avg_T_HA+2*weighted_std_T_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-3*weighted_std_T_HA, y2 = weighted_avg_T_HA+3*weighted_std_T_HA, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(framealpha=0.8)
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HE = np.average(loglog_param_HE[:, 0], weights=1/loglog_err_HE[:, 0]**2)
    weighted_std_T_HE = np.sqrt(np.average((loglog_param_HE[:, 0] - weighted_avg_T_HE)**2, weights=1/loglog_err_HE[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax2.errorbar(date_in_transit_HE, loglog_param_HE[:, 0], yerr = loglog_err_HE[:, 0], fmt='.', label='Retrieved')
    ax2.set_xlabel('Time (MJD)')
    ax2.set_ylabel('Temperature (K)')
    ax2.set_title('(Logfit) Best-fit temperature for HE')
    ax2.axhline(np.log10(temperature), color='k', label='Expected')
    ax2.axhline(weighted_avg_T_HE, color='r', label='Average')
    ax2.axhline(loglog_param_master_HE[0], color='b', label='Master')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-weighted_std_T_HE, y2 = weighted_avg_T_HE+weighted_std_T_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-2*weighted_std_T_HE, y2 = weighted_avg_T_HE+2*weighted_std_T_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-3*weighted_std_T_HE, y2 = weighted_avg_T_HE+3*weighted_std_T_HE, alpha=0.2, color='orange', label='3 $\sigma$')
    
    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HA = np.average(loglog_param_HA[:, 1], weights=1/loglog_err_HA[:, 1]**2)
    weighted_std_rho_HA = np.sqrt(np.average((loglog_param_HA[:, 1] - weighted_avg_rho_HA)**2, weights=1/loglog_err_HA[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax3.errorbar(date_in_transit_HA, loglog_param_HA[:, 1], yerr = loglog_err_HA[:, 1], fmt='.', label='Retrieved')
    ax3.set_xlabel('Time (MJD)')
    ax3.set_ylabel('Density ($m^{-3}$)')
    ax3.set_title('(Logfit) Best-fit column density for HA')
    ax3.axhline(np.log10(density), color='k', label='Expected')
    ax3.axhline(weighted_avg_rho_HA, color='r', label='Average')
    ax3.axhline(loglog_param_master_HA[1], color='b', label='Master')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-weighted_std_rho_HA, y2 = weighted_avg_rho_HA+weighted_std_rho_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-2*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+2*weighted_std_rho_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-3*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+3*weighted_std_rho_HA, alpha=0.2, color='orange', label='3 $\sigma$')

    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HE = np.average(loglog_param_HE[:, 1], weights=1/loglog_err_HE[:, 1]**2)
    weighted_std_rho_HE = np.sqrt(np.average((loglog_param_HE[:, 1] - weighted_avg_rho_HE)**2, weights=1/loglog_err_HE[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax4.errorbar(date_in_transit_HE, loglog_param_HE[:, 1], yerr = loglog_err_HE[:, 1], fmt='.', label='Retrieved ')
    ax4.set_xlabel('Time (MJD)')
    ax4.set_ylabel('Density ($m^{-3}$)')
    ax4.set_title('(Logfit) Best-fit column density for HE')
    ax4.axhline(np.log10(density), color='k', label='Expected')
    ax4.axhline(weighted_avg_rho_HE, color='r', label='Average')
    ax4.axhline(loglog_param_master_HE[1], color='b', label='Master')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-weighted_std_rho_HE, y2 = weighted_avg_rho_HE+weighted_std_rho_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-2*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+2*weighted_std_rho_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-3*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+3*weighted_std_rho_HE, alpha=0.2, color='orange', label='3 $\sigma$')

    ax1.sharey(ax2)
    ax3.sharey(ax4)

    fig.tight_layout()
    if save:
        plt.savefig(save_location+'Log_n_Log_T_Retrieved_Params.pdf')
    
    print('HA - Expected T is: '+str(np.log10(temperature))+' and retrieved T is: %.2f'%weighted_avg_T_HA,' and difference is: %.2f'%np.abs((np.log10(temperature) - weighted_avg_T_HA)/weighted_std_T_HA), ' sigma')
    print('HA - Expected density is: '+str(density/1e14)+' and retrieved density is: %.2f'%(10**weighted_avg_rho_HA / 1e14),' and difference is: %.2f'%np.abs((np.log10(density) - weighted_avg_rho_HA)/weighted_std_rho_HA), ' sigma')
    print('HE - Expected T is: '+str(np.log10(temperature))+' and retrieved T is: %.2f'%weighted_avg_T_HE,' and difference is: %.2f'%np.abs((np.log10(temperature) - weighted_avg_T_HE)/weighted_std_T_HE), ' sigma')
    print('HE - Expected density is: '+str(density/1e14)+' and retrieved density is: %.2f'%(10**weighted_avg_rho_HE / 1e14),' and difference is: %.2f'%np.abs((np.log10(density) - weighted_avg_rho_HE)/weighted_std_rho_HE), ' sigma')

#If there is one mode of observation
else:
    
    fig , (ax1, ax2) = plt.subplots(1, 2, figsize=[14, 8])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T = np.average(loglog_param[:, 0], weights=1/loglog_err[:, 0]**2)
    weighted_std_T = np.sqrt(np.average((loglog_param[:, 0] - weighted_avg_T)**2, weights=1/loglog_err[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit-59970, loglog_param[:, 0], yerr = loglog_err[:, 0], fmt='.', label='Retrieved')
    ax1.set_xlabel('Time (MJD-59970)', fontsize=14)
    ax1.set_ylabel('Log$_{10}$ Temperature (K)', fontsize=14)
    ax1.set_title('Atmospheric temperature', fontsize=14)
    ax1.axhline(np.log10(temperature), color='k', label='Expected')
    ax1.axhline(weighted_avg_T, color='r', label='Average')
    ax1.axhline(loglog_param_master[0], color='b', label='Master')
    ax1.tick_params(axis='both', labelsize=14)
    ax1.fill_between(date_in_transit-59970, y1 = weighted_avg_T-weighted_std_T, y2 = weighted_avg_T+weighted_std_T, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit-59970, y1 = weighted_avg_T-2*weighted_std_T, y2 = weighted_avg_T+2*weighted_std_T, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit-59970, y1 = weighted_avg_T-3*weighted_std_T, y2 = weighted_avg_T+3*weighted_std_T, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(fontsize=14, framealpha=.7, loc='upper right')

    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho = np.average(loglog_param[:, 1], weights=1/loglog_err[:, 1]**2)
    weighted_std_rho = np.sqrt(np.average((loglog_param[:, 1] - weighted_avg_rho)**2, weights=1/loglog_err[:, 1]**2))
        
    #Plotting the retrieved densities.
    ax2.errorbar(date_in_transit-59970, loglog_param[:, 1], yerr = loglog_err[:, 1], fmt='.', label='Retrieved')
    ax2.set_xlabel('Time (MJD-59970)', fontsize=14)
    ax2.set_ylabel('Log$_{10}$ Density ($cm^{-2}$)', fontsize=14)
    ax2.set_title('Helium column density', fontsize=14)
    ax2.axhline(np.log10(density), color='k', label='Expected')
    ax2.axhline(weighted_avg_rho, color='r', label='Average')
    ax2.axhline(loglog_param_master[1], color='b', label='Master')
    ax2.tick_params(axis='both', labelsize=14)
    ax2.fill_between(date_in_transit-59970, y1 = weighted_avg_rho-weighted_std_rho, y2 = weighted_avg_rho+weighted_std_rho, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit-59970, y1 = weighted_avg_rho-2*weighted_std_rho, y2 = weighted_avg_rho+2*weighted_std_rho, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit-59970, y1 = weighted_avg_rho-3*weighted_std_rho, y2 = weighted_avg_rho+3*weighted_std_rho, alpha=0.2, color='orange', label='3 $\sigma$')

    fig.tight_layout()
    if save:
        plt.savefig(save_location+'Log_n_Log_T_Retrieved_Params.pdf')
        
    print('Expected T is: '+str(np.log10(temperature))+' and retrieved T is: %.2f'%weighted_avg_T,' and difference is: %.2f'%np.abs((np.log10(temperature) - weighted_avg_T)/weighted_std_T), ' sigma')
    print('Expected density is: '+str(density/1e14)+' and retrieved density is: %.2f'%(10**weighted_avg_rho / 1e14),' and difference is: %.2f'%np.abs((np.log10(density) - weighted_avg_rho)/weighted_std_rho), ' sigma')


## Periodogram

In [ ]:
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    LS_T_HA = LombScargle(date_in_transit_HA, loglog_param_HA[:, 0], loglog_err_HA[:, 0]).autopower(nyquist_factor=1)
    LS_T_HE = LombScargle(date_in_transit_HE, loglog_param_HE[:, 0], loglog_err_HE[:, 0]).autopower(nyquist_factor=1)
    
    LS_rho_HA = LombScargle(date_in_transit_HA, loglog_param_HA[:, 1], loglog_err_HA[:, 1]).autopower(nyquist_factor=1)
    LS_rho_HE = LombScargle(date_in_transit_HE, loglog_param_HE[:, 1], loglog_err_HE[:, 1]).autopower(nyquist_factor=1)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=[10, 4])
    ax1.loglog(LS_T_HA[0], LS_T_HA[1])
    ax1.set_xlabel('Frequency (1/d)')
    ax1.set_ylabel('Power')
    ax1.set_title('Temperature Periodogram for HA')
    ax1.axvline(LS_T_HA[0][LS_T_HA[1]==np.max(LS_T_HA[1])])
    
    ax2.loglog(LS_T_HE[0], LS_T_HE[1])
    ax2.set_xlabel('Frequency (1/d)')
    ax2.set_title('Temperature Periodogram for HE')
    ax2.axvline(LS_T_HE[0][LS_T_HE[1]==np.max(LS_T_HE[1])])
    print(LS_T_HA[0][LS_T_HA[1]==np.max(LS_T_HA[1])], LS_T_HE[0][LS_T_HE[1]==np.max(LS_T_HE[1])])
    
    fig.tight_layout()
    plt.show()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=[10, 4])
    ax1.loglog(LS_rho_HA[0], LS_rho_HA[1])
    ax1.set_xlabel('Frequency (1/d)')
    ax1.set_ylabel('Power')
    ax1.set_title('Column Density Periodogram for HA')
    ax1.axvline(LS_rho_HA[0][LS_rho_HA[1]==np.max(LS_rho_HA[1])])
    
    ax2.loglog(LS_rho_HE[0], LS_rho_HE[1])
    ax2.set_xlabel('Frequency (1/d)')
    ax2.set_title('Column Density Periodogram for HE')
    ax2.axvline(LS_rho_HE[0][LS_rho_HE[1]==np.max(LS_rho_HE[1])])
    print(LS_rho_HA[0][LS_rho_HA[1]==np.max(LS_rho_HA[1])], LS_rho_HE[0][LS_rho_HE[1]==np.max(LS_rho_HE[1])])

    fig.tight_layout()
    plt.show()
    
    
else:

    LS_T = LombScargle(date_in_transit, loglog_param[:, 0], loglog_err[:, 0]).autopower(nyquist_factor=1)
    
    LS_rho = LombScargle(date_in_transit, loglog_param[:, 1], loglog_err[:, 1]).autopower(nyquist_factor=1)
    
    plt.loglog(LS_T[0], LS_T[1])
    plt.xlabel('Frequency (1/d)')
    plt.ylabel('Power')
    plt.title('Temperature Periodogram for HA')
    plt.axvline(LS_T[0][LS_T[1]==np.max(LS_T[1])])
    print(LS_T[0][LS_T[1]==np.max(LS_T[1])])
    
    plt.show()
    
    plt.loglog(LS_rho[0], LS_rho[1])
    plt.xlabel('Frequency (1/d)')
    plt.ylabel('Power')
    plt.title('Column Density Periodogram for HA')
    plt.axvline(LS_rho[0][LS_rho[1]==np.max(LS_rho[1])])
    print(LS_rho[0][LS_rho[1]==np.max(LS_rho[1])])

    plt.show()

## Check if there's a sinusoidal trend

In [ ]:
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    ## Double check that omega_rho and omega_T get overwritten when switching modes
    omega_rho_HA = LS_rho_HA[0][LS_rho_HA[1]==np.max(LS_rho_HA[1])][0]
    omega_T_HA = LS_T_HA[0][LS_T_HA[1]==np.max(LS_T_HA[1])][0]

    def sinusoid_T_HA(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_T_HA) + B*np.sin(2*np.pi*t*omega_T_HA)

    def sinusoid_rho_HA(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_rho_HA) + B*np.sin(2*np.pi*t*omega_rho_HA)

    loglog_param_T_HA, loglog_cov_T_HA =  curve_fit(sinusoid_T_HA, date_in_transit_HA, loglog_param_HA[:, 0], sigma = loglog_err_HA[:, 0])
    loglog_param_rho_HA, loglog_cov_rho_HA =  curve_fit(sinusoid_rho_HA, date_in_transit_HA, loglog_param_HA[:, 1], sigma = loglog_err_HA[:, 1], p0=[5e13, 5e13, 4.6e14])

    omega_rho_HE = LS_rho_HE[0][LS_rho_HE[1]==np.max(LS_rho_HE[1])][0]
    omega_T_HE = LS_T_HE[0][LS_T_HE[1]==np.max(LS_T_HE[1])][0]

    def sinusoid_T_HE(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_T_HE) + B*np.sin(2*np.pi*t*omega_T_HE)

    def sinusoid_rho_HE(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_rho_HE) + B*np.sin(2*np.pi*t*omega_rho_HE)

    loglog_param_T_HE, loglog_cov_T_HE =  curve_fit(sinusoid_T_HE, date_in_transit_HE, loglog_param_HE[:, 0], sigma = loglog_err_HE[:, 0])
    loglog_param_rho_HE, loglog_cov_rho_HE =  curve_fit(sinusoid_rho_HE, date_in_transit_HE, loglog_param_HE[:, 1], sigma = loglog_err_HE[:, 1], p0=[5e13, 5e13, 4.6e14])

else:
    
    omega_rho = LS_rho[0][LS_rho[1]==np.max(LS_rho[1])]
    omega_T = LS_T[0][LS_T[1]==np.max(LS_T[1])]
    
    def sinusoid_T(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_T) + B*np.sin(2*np.pi*t*omega_T)

    def sinusoid_rho(t, A, B, offset):
        return offset+A*np.cos(2*np.pi*t*omega_rho) + B*np.sin(2*np.pi*t*omega_rho)


    loglog_param_T, loglog_cov_T =  curve_fit(sinusoid_T, date_in_transit, loglog_param[:, 0], sigma = loglog_err[:, 0])
    loglog_param_rho, loglog_cov_rho =  curve_fit(sinusoid_rho, date_in_transit, loglog_param[:, 1], sigma = loglog_err[:, 1], p0=[5e13, 5e13, 4.6e14])

In [ ]:
reload(AF)

#Defining a variable to save or not the results.
save = True

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=[18, 12])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HA = np.average(loglog_param_HA[:, 0], weights=1/loglog_err_HA[:, 0]**2)
    weighted_std_T_HA = np.sqrt(np.average((loglog_param_HA[:, 0] - weighted_avg_T_HA)**2, weights=1/loglog_err_HA[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit_HA, loglog_param_HA[:, 0], yerr = loglog_err_HA[:, 0], fmt='.', label='Retrieved')
    ax1.plot(date_in_transit_HA, sinusoid_T_HA(date_in_transit_HA, loglog_param_T_HA[0], loglog_param_T_HA[1], loglog_param_T_HA[2]), 'm', label='Model')
    ax1.set_xlabel('Time (MJD)')
    ax1.set_ylabel('Temperature (K)')
    ax1.set_title('(Logfit) Best-fit temperature for HA')
    ax1.axhline(np.log10(temperature), color='k', label='Expected')
    ax1.axhline(weighted_avg_T_HA, color='r', label='Average')
    ax1.axhline(loglog_param_master_HA[0], color='b', label='Master')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-weighted_std_T_HA, y2 = weighted_avg_T_HA+weighted_std_T_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-2*weighted_std_T_HA, y2 = weighted_avg_T_HA+2*weighted_std_T_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit_HA, y1 = weighted_avg_T_HA-3*weighted_std_T_HA, y2 = weighted_avg_T_HA+3*weighted_std_T_HA, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(framealpha=0.8)
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T_HE = np.average(loglog_param_HE[:, 0], weights=1/loglog_err_HE[:, 0]**2)
    weighted_std_T_HE = np.sqrt(np.average((loglog_param_HE[:, 0] - weighted_avg_T_HE)**2, weights=1/loglog_err_HE[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax2.errorbar(date_in_transit_HE, loglog_param_HE[:, 0], yerr = loglog_err_HE[:, 0], fmt='.', label='Retrieved')
    ax2.plot(date_in_transit_HE, sinusoid_T_HE(date_in_transit_HE, loglog_param_T_HE[0], loglog_param_T_HE[1], loglog_param_T_HE[2]), 'm', label='Model')
    ax2.set_xlabel('Time (MJD)')
    ax2.set_ylabel('Temperature (K)')
    ax2.set_title('(Logfit) Best-fit temperature for HE')
    ax2.axhline(np.log10(temperature), color='k', label='Expected')
    ax2.axhline(weighted_avg_T_HE, color='r', label='Average')
    ax2.axhline(loglog_param_master_HE[0], color='b', label='Master')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-weighted_std_T_HE, y2 = weighted_avg_T_HE+weighted_std_T_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-2*weighted_std_T_HE, y2 = weighted_avg_T_HE+2*weighted_std_T_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit_HE, y1 = weighted_avg_T_HE-3*weighted_std_T_HE, y2 = weighted_avg_T_HE+3*weighted_std_T_HE, alpha=0.2, color='orange', label='3 $\sigma$')
    
    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HA = np.average(loglog_param_HA[:, 1], weights=1/loglog_err_HA[:, 1]**2)
    weighted_std_rho_HA = np.sqrt(np.average((loglog_param_HA[:, 1] - weighted_avg_rho_HA)**2, weights=1/loglog_err_HA[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax3.errorbar(date_in_transit_HA, loglog_param_HA[:, 1], yerr = loglog_err_HA[:, 1], fmt='.', label='Retrieved')
    ax3.plot(date_in_transit_HA, sinusoid_rho_HA(date_in_transit_HA, loglog_param_rho_HA[0], loglog_param_rho_HA[1], loglog_param_rho_HA[2]), 'm', label='Model')
    ax3.set_xlabel('Time (MJD)')
    ax3.set_ylabel('Density ($m^{-3}$)')
    ax3.set_title('(Logfit) Best-fit column density for HA')
    ax3.axhline(np.log10(density), color='k', label='Expected')
    ax3.axhline(weighted_avg_rho_HA, color='r', label='Average')
    ax3.axhline(loglog_param_master_HA[1], color='b', label='Master')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-weighted_std_rho_HA, y2 = weighted_avg_rho_HA+weighted_std_rho_HA, alpha=0.6, color='orange', label='1 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-2*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+2*weighted_std_rho_HA, alpha=0.4, color='orange', label='2 $\sigma$')
    ax3.fill_between(date_in_transit_HA, y1 = weighted_avg_rho_HA-3*weighted_std_rho_HA, y2 = weighted_avg_rho_HA+3*weighted_std_rho_HA, alpha=0.2, color='orange', label='3 $\sigma$')

    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho_HE = np.average(loglog_param_HE[:, 1], weights=1/loglog_err_HE[:, 1]**2)
    weighted_std_rho_HE = np.sqrt(np.average((loglog_param_HE[:, 1] - weighted_avg_rho_HE)**2, weights=1/loglog_err_HE[:, 1]**2))
    
    #Plotting the retrieved densities.
    ax4.errorbar(date_in_transit_HE, loglog_param_HE[:, 1], yerr = loglog_err_HE[:, 1], fmt='.', label='Retrieved ')
    ax4.plot(date_in_transit_HE, sinusoid_rho_HE(date_in_transit_HE, loglog_param_rho_HE[0], loglog_param_rho_HE[1], loglog_param_rho_HE[2]), 'm', label='Model')
    ax4.set_xlabel('Time (MJD)')
    ax4.set_ylabel('Density ($m^{-3}$)')
    ax4.set_title('(Logfit) Best-fit column density for HE')
    ax4.axhline(np.log10(density), color='k', label='Expected')
    ax4.axhline(weighted_avg_rho_HE, color='r', label='Average')
    ax4.axhline(loglog_param_master_HE[1], color='b', label='Master')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-weighted_std_rho_HE, y2 = weighted_avg_rho_HE+weighted_std_rho_HE, alpha=0.6, color='orange', label='1 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-2*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+2*weighted_std_rho_HE, alpha=0.4, color='orange', label='2 $\sigma$')
    ax4.fill_between(date_in_transit_HE, y1 = weighted_avg_rho_HE-3*weighted_std_rho_HE, y2 = weighted_avg_rho_HE+3*weighted_std_rho_HE, alpha=0.2, color='orange', label='3 $\sigma$')

    ax1.sharey(ax2)
    ax3.sharey(ax4)

    fig.tight_layout()
    if save:
        plt.savefig(location+'/Plot_Results/Log_Sin_Retrieved_Params.pdf')
        
        
    fig, (ax1, ax2)

#If there is one mode of observation
else:
    
    fig , (ax1, ax2) = plt.subplots(1, 2, figsize=[14, 6])
    
    #Defining the average and standard deviation of the retrieved temperatures.
    weighted_avg_T = np.average(loglog_param[:, 0], weights=1/loglog_err[:, 0]**2)
    weighted_std_T = np.sqrt(np.average((loglog_param[:, 0] - weighted_avg_T)**2, weights=1/loglog_err[:, 0]**2))
    
    #Plotting the retrieved temperatures.
    ax1.errorbar(date_in_transit, loglog_param[:, 0], yerr = loglog_err[:, 0], fmt='.', label='Retrieved')
    ax1.plot(date_in_transit, sinusoid_T(date_in_transit, loglog_param_T[0], loglog_param_T[1], loglog_param_T[2]), 'm', label='Model')
    ax1.set_xlabel('Time (MJD)')
    ax1.set_ylabel('Temperature (K)')
    ax1.set_title('(Logfit) Best-fit temperature')
    ax1.axhline(np.log10(temperature), color='k', label='Expected')
    ax1.axhline(weighted_avg_T, color='r', label='Average')
    ax1.axhline(loglog_param_master[0], color='b', label='Master')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-weighted_std_T, y2 = weighted_avg_T+weighted_std_T, alpha=0.6, color='orange', label='1 $\sigma$')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-2*weighted_std_T, y2 = weighted_avg_T+2*weighted_std_T, alpha=0.4, color='orange', label='2 $\sigma$')
    ax1.fill_between(date_in_transit, y1 = weighted_avg_T-3*weighted_std_T, y2 = weighted_avg_T+3*weighted_std_T, alpha=0.2, color='orange', label='3 $\sigma$')
    ax1.legend(framealpha=.7)

    #Defining the average and standard deviation of the retrieved densities.
    weighted_avg_rho = np.average(loglog_param[:, 1], weights=1/loglog_err[:, 1]**2)
    weighted_std_rho = np.sqrt(np.average((loglog_param[:, 1] - weighted_avg_rho)**2, weights=1/loglog_err[:, 1]**2))
        
    #Plotting the retrieved densities.
    ax2.errorbar(date_in_transit, loglog_param[:, 1], yerr = loglog_err[:, 1], fmt='.', label='Retrieved')
    ax2.plot(date_in_transit, sinusoid_rho(date_in_transit, loglog_param_rho[0], loglog_param_rho[1], loglog_param_rho[2]), 'm', label='Model')
    ax2.set_xlabel('Time (MJD)')
    ax2.set_ylabel('Density ($m^{-3}$)')
    ax2.set_title('(Logfit) Best-fit column density')
    ax2.axhline(np.log10(density), color='k', label='Expected')
    ax2.axhline(weighted_avg_rho, color='r', label='Average')
    ax2.axhline(loglog_param_master[1], color='b', label='Master')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-weighted_std_rho, y2 = weighted_avg_rho+weighted_std_rho, alpha=0.6, color='orange', label='1 $\sigma$')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-2*weighted_std_rho, y2 = weighted_avg_rho+2*weighted_std_rho, alpha=0.4, color='orange', label='2 $\sigma$')
    ax2.fill_between(date_in_transit, y1 = weighted_avg_rho-3*weighted_std_rho, y2 = weighted_avg_rho+3*weighted_std_rho, alpha=0.2, color='orange', label='3 $\sigma$')

    fig.tight_layout()
    if save:
        plt.savefig(location+'/Plot_Results/Log_Sin_Retrieved_Params.pdf')

# Large-scale analysis

## Big chunk (data generation + fitting)

In [ ]:
reload(AF)

T_s = np.linspace(5000, 20000, 20)
n_s = np.logspace(14, 15.5, 20)

plot_true = True

using_Rassine = True

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    w_av_T_HA = []
    w_av_rho_HA = []
    w_av_T_HE = []
    w_av_rho_HE = []
    T_master_HA = []
    T_master_HE = []
    rho_master_HA = []
    rho_master_HE = []
    
else:
    w_av_T = []
    w_av_rho = []
    T_master = []
    rho_master = []

for t in range(len(T_s)):
    temperature = T_s[t]
    density = n_s[t]
    #################################################################################
    ####Adding the planetary signal during transit###################################
    #################################################################################

    #Distinguish two cases depending on the number of modes of observation
    #If there are two modes of observation
    if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
        #Initiating arrays to contain the in-transit spectra with a planetary signal and their corresponding errors.
        planet_in_transit_spctr_HA = np.ones(in_transit_spctr_HA.shape)
        planet_in_transit_err_HA = np.ones(in_transit_err_HA.shape)

        planet_in_transit_spctr_HE = np.ones(in_transit_spctr_HE.shape)
        planet_in_transit_err_HE = np.ones(in_transit_err_HE.shape)

        #Populating the above defined arrays.
        wav_to_use = rassine_total_lamda_HA[0]
        for i in range(len(in_transit_spctr_HA)):

            planet_in_transit_spctr_HA[i] = spec_mod.spectrum_he(wav_to_use, in_transit_spctr_HA[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HA)
            planet_in_transit_err_HA[i] = spec_mod.spectrum_he(wav_to_use, in_transit_err_HA[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HA)

        wav_to_use = rassine_total_lamda_HE[0]
        for i in range(len(in_transit_spctr_HE)):

            planet_in_transit_spctr_HE[i] = spec_mod.spectrum_he(wav_to_use, in_transit_spctr_HE[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HE)
            planet_in_transit_err_HE[i] = spec_mod.spectrum_he(wav_to_use, in_transit_err_HE[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HE)

    #If there is one mode of observation    
    else:
        #Initiating arrays to contain the in-transit spectra with a planetary signal and their corresponding errors.
        planet_in_transit_spctr = np.ones(in_transit_spctr.shape)
        planet_in_transit_err = np.ones(in_transit_err.shape)

        wav_to_use = rassine_total_lamda[0]
        #Populating the above defined arrays.
        for i in range(len(in_transit_spctr)):

            planet_in_transit_spctr[i] = spec_mod.spectrum_he(wav_to_use, in_transit_spctr[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power)
            planet_in_transit_err[i] = spec_mod.spectrum_he(wav_to_use, in_transit_err[i], temperature, density, vsini, medium, RV_offset_He, R_pow = R_power)

    
    #################################################################################
    ####Making the master spectra ###################################################
    #################################################################################
    
    #Distinguish two cases depending on the number of modes of observation
    #If there are two modes of observation
    if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
        #Making the master spectra, in and out of transit.
        if using_Rassine:
            #In-transit
            master_planet_intransit_HA = spec_mod.spectrum_he(rassine_master_lamda_HA, rassine_master_spctr_HA, temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HA)
            master_err_intransit_HA = np.zeros(master_planet_intransit_HA.shape)

            master_planet_intransit_HE = spec_mod.spectrum_he(rassine_master_lamda_HE, rassine_master_spctr_HE, temperature, density, vsini, medium, RV_offset_He, R_pow = R_power_HE)
            master_err_intransit_HE = np.zeros(master_planet_intransit_HE.shape)

            #Out-of-transit 
            master_outtransit_HA = rassine_master_spctr_HA
            master_err_outtransit_HA = np.zeros(rassine_master_spctr_HA.shape)

            master_outtransit_HE = rassine_master_spctr_HE
            master_err_outtransit_HE = np.zeros(rassine_master_spctr_HE.shape)

        else:
            #In-transit
            master_planet_intransit_HA = np.average(planet_in_transit_spctr_HA, axis=0, weights = 1/planet_in_transit_err_HA**2)
            master_err_intransit_HA = 1/np.sqrt(np.sum(1/planet_in_transit_err_HA**2, axis=0))

            master_planet_intransit_HE = np.average(planet_in_transit_spctr_HE, axis=0, weights = 1/planet_in_transit_err_HE**2)
            master_err_intransit_HE = 1/np.sqrt(np.sum(1/planet_in_transit_err_HE**2, axis=0))

            #Out-of-transit
            master_outtransit_HA = np.average(out_transit_spctr_HA, axis=0, weights = 1/out_transit_err_HA**2)
            master_err_outtransit_HA = 1/np.sqrt(np.sum(1/out_transit_err_HA**2, axis=0))

            master_outtransit_HE = np.average(out_transit_spctr_HE, axis=0, weights = 1/out_transit_err_HE**2)
            master_err_outtransit_HE = 1/np.sqrt(np.sum(1/out_transit_err_HE**2, axis=0))

    #If there is one mode of observation
    else:
        #Making the master spectra, in and out of transit.
        
        if using_Rassine:
            #In-transit
            master_planet_intransit = spec_mod.spectrum_he(rassine_master_lamda, rassine_master_spctr, temperature, density, vsini, medium, RV_offset_He, R_pow = R_power)
            master_err_intransit = np.zeros(master_planet_intransit.shape)

            #Out-of-transit 
            master_outtransit = rassine_master_spctr
            master_err_outtransit = np.zeros(rassine_master_spctr.shape)

        else:
            #In-transit
            master_planet_intransit = np.average(planet_in_transit_spctr, axis=0, weights = 1/planet_in_transit_err**2)
            master_err_intransit = 1/np.sqrt(np.sum(1/planet_in_transit_err**2, axis=0))

            #Out-of-transit
            master_outtransit = np.average(out_transit_spctr, axis=0, weights = 1/out_transit_err**2)
            master_err_outtransit = 1/np.sqrt(np.sum(1/out_transit_err**2, axis=0))


        
    #################################################################################
    ####Making the planetary absorption spectra #####################################
    #################################################################################
  
    ##Defining the difference between the in-transit and master out-of-transit spectra to get 
    #a time series of the planet's spectrum
    offset = 1

    #Distinguish two cases depending on the number of modes of observation
    #If there are two modes of observation
    if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

        #Initiating arrays to contain the values and error of the planetary absorption spectrum.
        diff_HA = np.ones(planet_in_transit_spctr_HA.shape)
        diff_HA_err = np.ones(planet_in_transit_err_HA.shape)

        #Populating the above defined arrays.
        #We add an offset of 1 for later fitting purposes.
        for i in range(len(planet_in_transit_spctr_HA)):
            diff_HA[i] = offset+(master_outtransit_HA - planet_in_transit_spctr_HA[i])/master_outtransit_HA
            diff_HA_err[i] = np.sqrt((planet_in_transit_err_HA[i]/master_outtransit_HA)**2 + ((planet_in_transit_spctr_HA[i] * master_err_outtransit_HA)/(master_outtransit_HA**2))**2)

        diff_HE = np.ones(planet_in_transit_spctr_HE.shape)
        diff_HE_err = np.ones(planet_in_transit_err_HE.shape)
        for i in range(len(planet_in_transit_spctr_HE)):
            diff_HE[i] = offset+(master_outtransit_HE - planet_in_transit_spctr_HE[i])/master_outtransit_HE
            diff_HE_err[i] = np.sqrt((planet_in_transit_err_HE[i]/master_outtransit_HE)**2 + ((planet_in_transit_spctr_HE[i] * master_err_outtransit_HE)/(master_outtransit_HE**2))**2)

    #If there is one mode of observation
    else:

        #Initiating arrays to contain the values and error of the planetary absorption spectrum.
        diff = np.ones(planet_in_transit_spctr.shape)
        diff_err = np.ones(planet_in_transit_err.shape)

        #Populating the above defined arrays.
        #We add an offset of 1 for later fitting purposes.
        for i in range(len(planet_in_transit_spctr)):
            diff[i] = offset+(master_outtransit - planet_in_transit_spctr[i])/master_outtransit
            diff_err[i] = np.sqrt((planet_in_transit_err[i]/master_outtransit)**2 + ((planet_in_transit_spctr[i] * master_err_outtransit)/(master_outtransit**2))**2)

    #################################################################################
    ####Retrieving the planetary parameters #########################################
    #################################################################################

    #Distinguish two cases depending on the number of modes of observation
    #If there are two modes of observation
    if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

        #Re-defining the absorption function
        #We fix the stellar velocity because the Sun's velocity is well known.
        #We fit for the atmospheric He column density and the temperature of the atmosphere.
        def absorption_function_HA(x, temp, log_density, offset):
            return spec_mod.spectrum_he(x, offset+np.zeros(diff_HA[0].shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HA)

        def absorption_function_HE(x, temp, log_density, offset):
            return spec_mod.spectrum_he(x, offset+np.zeros(diff_HE[0].shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HE)

        #Creating arrays to store the best-fit parameters and the error on them
        param_HA = np.zeros((len(diff_HA), 3))
        err_HA = np.zeros((len(diff_HA), 3))

        param_HE = np.zeros((len(diff_HE), 3))
        err_HE = np.zeros((len(diff_HE), 3))

        ref_wave = rassine_total_lamda_HA[0]
        #Iterating over the time series to get a value of the retrieved parameters for each spectrum
        for i in range(len(diff_HA)):

            guess = [temperature+1000, np.log10(density)+0.2, np.median(diff_HA[i])]

            #Chi-squared minimization routine
            params, cov = curve_fit(absorption_function_HA, ref_wave, diff_HA[i], sigma = diff_HA_err[i], p0=guess)

            #Storing the best-fit parameters and errors
            param_HA[i] = params
            err_HA[i] = np.sqrt(np.diag(cov))

            #Plotting the in-transit planetary spectra and their corresponding model
            
            model = absorption_function_HA(ref_wave, params[0], params[1], params[2])
            if plot_true:
                plt.figure(figsize=[10, 6])
                plt.errorbar(ref_wave, 100*(diff_HA[i]-offset), yerr=100*diff_HA_err[i], fmt='b.', label='Simulated data')
                plt.plot(ref_wave, 100*(absorption_function_HA(ref_wave, temperature, np.log10(density), params[2])-offset), color='orange', label='Expected result')
                plt.plot(ref_wave, 100*(model-offset), 'r', label='Best-fit')
                plt.xlim([10825, 10840])
                plt.xlabel('Wavelength ($\AA$)')
                plt.ylabel('Absorption (%)')
                plt.ylim(-2, 7)
                plt.legend()
                plt.show()

        ref_wave = rassine_total_lamda_HE[0]
        for i in range(len(diff_HE)):

            #Guess on the initial parameters
            guess = [temperature+1000, np.log10(density)+0.2, np.median(diff_HE[i])]

            bnds = ((1000, 13, -np.inf), (30000, 16, np.inf))

            #Chi-squared minimization routine
            params, cov = curve_fit(absorption_function_HE, ref_wave, diff_HE[i], sigma=diff_HE_err[i], p0=guess, bounds=bnds)

            #Storing the best-fit parameters and errors
            param_HE[i] = params
            err_HE[i] = np.sqrt(np.diag(cov))

            #Plotting the in-transit planetary spectra and their corresponding model
            
            model = absorption_function_HE(ref_wave, params[0], params[1], params[2])
            
            if plot_true:
                plt.figure(figsize=[10, 6])
                plt.errorbar(ref_wave, 100*(diff_HE[i]-offset), yerr = 100*diff_HE_err[i], fmt='b.', label='Simulated data')
                plt.plot(ref_wave, 100*(absorption_function_HE(ref_wave, temperature, np.log10(density), params[2])-offset), color='orange', label='Expected result')
                plt.plot(ref_wave, 100*(model-offset), 'k', label='Best-fit')
                plt.xlim([10825, 10840])
                plt.xlabel('Wavelength ($\AA$)')
                plt.ylabel('Absorption (%)')
                plt.ylim(-2, 7)
                plt.legend()
                plt.show()

    #If there is one mode of observation
    else:
        #Re-defining the absorption function
        #We fix the stellar velocity because the Sun's velocity is well known.
        #We fit for the atmospheric He column density and the temperature of the atmosphere.
        def absorption_function(x, temp, log_density, offset):
            return spec_mod.spectrum_he(x, offset+np.zeros(diff[0].shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power)

        #Creating arrays to store the best-fit parameters and the error on them
        param_total = np.zeros((len(diff), 3))
        err_total = np.zeros((len(diff), 3))

        ref_wave = rassine_total_lamda[0]
        #Iterating over the time series to get a value of the retrieved parameters for each spectrum
        for i in range(len(diff)):

            guess = [temperature+1000, np.log10(density)+0.2, np.median(diff[i])]

            bnds = ((1000, 14, -np.inf), (22000, 16, np.inf))

            #Chi-squared minimization routine
            params, cov = curve_fit(absorption_function, ref_wave, diff[i], sigma=diff_err[i], p0=guess, bounds = bnds)

            #Storing the best-fit parameters and errors
            param_total[i] = params
            err_total[i] = np.sqrt(np.diag(cov))

            #Plotting the in-transit planetary spectra and their corresponding model
            model = absorption_function(ref_wave, params[0], params[1], params[2])
            
            if plot_true:
                plt.figure(figsize=[10, 6])
                plt.errorbar(ref_wave, 100*(diff[i]-offset), 100*diff_err[i], fmt='b.', label='Simulated data')
                plt.plot(ref_wave, 100*(absorption_function(ref_wave, temperature, np.log10(density), params[2])-offset), color='orange', label='Expected result')
                plt.plot(ref_wave, 100*(model-offset), 'r', label='Best-fit')
                plt.xlim([10825, 10840])
                plt.xlabel('Wavelength ($\AA$)')
                plt.ylabel('Absorption (%)')
                plt.legend()
                plt.show()

                
    #################################################################################
    ####Double checking with master absorption spectrum fit #########################
    #################################################################################

    
    #Distinguish two cases depending on the number of modes of observation
    #If there are two modes of observation
    if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

        #HA
        diff_master_HA = offset+(master_outtransit_HA - master_planet_intransit_HA)/master_outtransit_HA

        diff_master_err_HA = np.sqrt((master_err_intransit_HA/master_outtransit_HA)**2 + ((master_planet_intransit_HA * master_err_outtransit_HA)/(master_outtransit_HA**2))**2)

        ref_wave = rassine_master_lamda_HA
        
        def absorption_function_HA(x, temp, log_density, offset):
            return spec_mod.spectrum_he(x, offset+np.zeros(diff_master_HA.shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HA)

        guess = [temperature+1000, np.log10(density)+0.5, np.median(diff_master_HA)]
        
        #Chi-squared minimization routine
        if using_Rassine:
            params, cov = curve_fit(absorption_function_HA, ref_wave, diff_master_HA, p0=guess)
        else:
            params, cov = curve_fit(absorption_function_HA, ref_wave, diff_master_HA, sigma=diff_master_err_HA, p0=guess)
            
        #Storing the best-fit parameters and errors
        param_master_HA = params
        err_master_HA = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        
        plt.figure(figsize=[10, 6])
        model = absorption_function_HA(ref_wave, params[0], params[1], params[2])
        
        if plot_true:
            plt.errorbar(ref_wave, 100*(diff_master_HA-offset), 100*diff_master_err_HA, fmt='b.', label='Simulated data')
            plt.plot(ref_wave, 100*(absorption_function_HA(ref_wave, temperature, np.log10(density), params[2])-offset), color='orange', label='Expected result')
            plt.plot(ref_wave, 100*(model-offset), 'r', label='Best-fit')
            plt.xlim([10825, 10840])
            plt.xlabel('Wavelength ($\AA$)')
            plt.ylabel('Absorption (%)')
            plt.legend()
            plt.show()

        T_master_HA.append(param_master_HA[0])
        rho_master_HA.append(param_master_HA[1])

        #HE
        diff_master_HE = offset+(master_outtransit_HE - master_planet_intransit_HE)/master_outtransit_HE

        diff_master_err_HE = np.sqrt((master_err_intransit_HE/master_outtransit_HE)**2 + ((master_planet_intransit_HE * master_err_outtransit_HE)/(master_outtransit_HE**2))**2)

        ref_wave = rassine_master_lamda_HE
        
        def absorption_function_HE(x, temp, log_density, offset):
            return spec_mod.spectrum_he(x, offset+np.zeros(diff_master_HE.shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power_HE)

        guess = [temperature+1000, np.log10(density)+0.5, np.median(diff_master_HE)]

        #Chi-squared minimization routine
        if using_Rassine:
            params, cov = curve_fit(absorption_function_HE, ref_wave, diff_master_HE, p0=guess)
        else:
            params, cov = curve_fit(absorption_function_HE, ref_wave, diff_master_HE, sigma=diff_master_err_HE, p0=guess)
 
        #Storing the best-fit parameters and errors
        param_master_HE = params
        err_master_HE = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        
        plt.figure(figsize=[10, 6])
        model = absorption_function_HE(ref_wave, params[0], params[1], params[2])
        
        if plot_true:
            plt.errorbar(ref_wave, 100*(diff_master_HE-offset), 100*diff_master_err_HE, fmt='b.', label='Simulated data')
            plt.plot(ref_wave, 100*(absorption_function_HE(ref_wave, temperature, np.log10(density), params[2])-offset), color='orange', label='Expected result')
            plt.plot(ref_wave, 100*(model-offset), 'k', label='Best-fit')
            plt.xlim([10825, 10840])
            plt.xlabel('Wavelength ($\AA$)')
            plt.ylabel('Absorption (%)')
            plt.legend()
            plt.show()

        T_master_HE.append(param_master_HE[0])
        rho_master_HE.append(param_master_HE[1])

    else:
        ref_wav = rassine_master_lamda
        
        diff_master = offset+(master_outtransit - master_planet_intransit)/master_outtransit

        diff_master_err = np.sqrt((master_err_intransit/master_outtransit)**2 + ((master_planet_intransit * master_err_outtransit)/(master_outtransit**2))**2)

        def absorption_function(x, temp, log_density, offset):
            return spec_mod.spectrum_he(x, offset+np.zeros(diff_master.shape), temp, -10**log_density, vsini, 'VACUUM', RV_offset_He, R_pow = R_power)

        guess = [temperature+1000, np.log10(density)+0.5, np.median(diff_master)]

        #Chi-squared minimization routine
        if using_Rassine:
            params, cov = curve_fit(absorption_function, ref_wave, diff_master, p0=guess)
        else:
            params, cov = curve_fit(absorption_function, ref_wave, diff_master, sigma=diff_master_err, p0=guess)
            
        #Storing the best-fit parameters and errors
        param_master = params
        err_master = np.sqrt(np.diag(cov))

        #Plotting the in-transit planetary spectra and their corresponding model
        
        model = absorption_function(ref_wave, params[0], params[1], params[2])

        if True:
            plt.figure(figsize=[10, 6])
            plt.errorbar(ref_wave, 100*(diff_master-offset), 100*diff_master_err, fmt='b.', label='Simulated data')
            plt.plot(ref_wave, 100*(absorption_function(ref_wave, temperature, np.log10(density), params[2])-offset), color='orange', label='Expected result')
            plt.plot(ref_wave, 100*(model-offset), 'r', label='Best-fit')
            plt.xlim([10825, 10840])
            plt.xlabel('Wavelength ($\AA$)')
            plt.ylabel('Absorption (%)')
            plt.legend()
            plt.show()
        
        T_master.append(param_master[0])
        rho_master.append(param_master[1])
            
    #################################################################################
    ####Plotting the results ########################################################
    #################################################################################
    
    #Distinguish two cases depending on the number of modes of observation
    #If there are two modes of observation
    if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
        
        #Defining the average and standard deviation of the retrieved temperatures.
        weighted_avg_T_HA = np.average(param_HA[:, 0], weights=1/err_HA[:, 0]**2)
        w_av_T_HA.append(weighted_avg_T_HA)
        
        #Defining the average and standard deviation of the retrieved temperatures.
        weighted_avg_T_HE = np.average(param_HE[:, 0], weights=1/err_HE[:, 0]**2)
        w_av_T_HE.append(weighted_avg_T_HE)
        
        #Defining the average and standard deviation of the retrieved densities.
        weighted_avg_rho_HA = np.average(param_HA[:, 1], weights=1/err_HA[:, 1]**2)
        w_av_rho_HA.append(weighted_avg_rho_HA)
        
        #Defining the average and standard deviation of the retrieved densities.
        weighted_avg_rho_HE = np.average(param_HE[:, 1], weights=1/err_HE[:, 1]**2)
        w_av_rho_HE.append(weighted_avg_rho_HE)
        
    #If there is one mode of observation
    else:
        
        #Defining the average and standard deviation of the retrieved temperatures.
        weighted_avg_T = np.average(param_total[:, 0], weights=1/err_total[:, 0]**2)
        w_av_T.append(weighted_avg_T)
        
        #Defining the average and standard deviation of the retrieved densities.
        weighted_avg_rho = np.average(param_total[:, 1], weights=1/err_total[:, 1]**2)
        w_av_rho.append(weighted_avg_rho)
           

## Plotting

In [ ]:
#Transforming our lists into arrays
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    w_av_T_HA = np.array(w_av_T_HA)
    w_av_rho_HA = np.array(w_av_rho_HA)
    w_av_T_HE = np.array(w_av_T_HE)
    w_av_rho_HE = np.array(w_av_rho_HE)
    T_master_HA = np.array(T_master_HA)
    T_master_HE = np.array(T_master_HE)
    rho_master_HA = np.array(rho_master_HA)
    rho_master_HE = np.array(rho_master_HE)
    
else:
    w_av_T = np.array(w_av_T)
    w_av_rho = np.array(w_av_rho)
    T_master = np.array(T_master)
    rho_master = np.array(rho_master)


In [ ]:
save=True

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #HA
    fig = plt.figure(figsize=[15, 10])
    # Create the grid of subplots using subplot_mosaic
    grid = fig.subplot_mosaic(
        [["main", "right"], 
         ["bottom", "bottom2"]],
        gridspec_kw={"height_ratios": [3, 1], "width_ratios": [3, 1]}
    )

    # Plot in the main plot
    ax_main = grid["main"]
    ax_main.plot(np.log10(n_s), T_s, 'ks', label='Expected', linestyle='-', markersize=12)
    ax_main.plot(w_av_rho_HA, w_av_T_HA, 'r^', label='Weighted Average', linestyle='-', markersize=12)
    ax_main.plot(rho_master_HA, T_master_HA, 'b.', label='Master', linestyle='-', markersize=12)
    ax_main.set_ylabel('Temperature (K)', fontsize=12)
    ax_main.tick_params(labelsize=12)
    ax_main.legend(fontsize=12)
    ax_main.set_title('Retrieved Parameter Grid - HA', fontsize=12)
    
    # Plot in the subplot to the right
    ax_right = grid["right"]
    ax_right.plot(w_av_rho_HA - np.log10(n_s), w_av_T_HA, 'r^', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(w_av_rho_HA - np.log10(n_s)))
    ax_right.plot(rho_master_HA - np.log10(n_s), T_master_HA, 'b.', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(rho_master_HA - np.log10(n_s)))
    ax_right.plot((rho_master_HA - np.log10(n_s))*0, T_master_HA, 'k', linestyle='--')
    ax_right.yaxis.tick_right()
    ax_right.tick_params(labelsize=12)
    ax_right.set_xlabel('Density Residuals (m$^{-2}$)', fontsize=12)
    ax_right.legend(fontsize=12)

    # Plot in the subplot below
    ax_bottom = grid["bottom"]
    ax_bottom.plot(w_av_rho_HA, w_av_T_HA - T_s, 'r^', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(w_av_T_HA - T_s))
    ax_bottom.plot(rho_master_HA, T_master_HA - T_s, 'b.', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(T_master_HA - T_s))
    ax_bottom.plot(rho_master_HA, (T_master_HA - T_s)*0, 'k', linestyle='--')
    ax_bottom.sharex(grid["main"])
    ax_bottom.set_xlabel('Log column density (m$^{-2}$)', fontsize=12)
    ax_bottom.set_ylabel('Temp. Residuals (K)', fontsize=12)
    ax_bottom.tick_params(labelsize=12)
    ax_bottom.legend(fontsize=12)
    
    fig.delaxes(grid['bottom2'])
    # Adjust the spacing between subplots
    fig.subplots_adjust(wspace=0, hspace=0)
    if save:
        plt.savefig(save_location+'Retrieved_param_big_HA.pdf')
    
    #HE
    fig = plt.figure(figsize=[15, 10])
    # Create the grid of subplots using subplot_mosaic
    grid = fig.subplot_mosaic(
        [["main", "right"], 
         ["bottom", "bottom2"]],
        gridspec_kw={"height_ratios": [3, 1], "width_ratios": [3, 1]}
    )

    # Plot in the main plot
    ax_main = grid["main"]
    ax_main.plot(np.log10(n_s), T_s, 'ks', label='Expected', linestyle='-', markersize=12)
    ax_main.plot(w_av_rho_HE, w_av_T_HE, 'r^', label='Weighted Average', linestyle='-', markersize=12)
    ax_main.plot(rho_master_HE, T_master_HE, 'b.', label='Master', linestyle='-', markersize=12)
    ax_main.set_ylabel('Temperature (K)', fontsize=12)
    ax_main.tick_params(labelsize=12)
    ax_main.legend(fontsize=12)
    ax_main.set_title('Retrieved Parameter Grid - HE', fontsize=12)
    
    # Plot in the subplot to the right
    ax_right = grid["right"]
    ax_right.plot(w_av_rho_HE - np.log10(n_s), w_av_T_HE, 'r^', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(w_av_rho_HE - np.log10(n_s)))
    ax_right.plot(rho_master_HE - np.log10(n_s), T_master_HE, 'b.', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(rho_master_HE - np.log10(n_s)))
    ax_right.plot((rho_master_HE - np.log10(n_s))*0, T_master_HE, 'k', linestyle='--')
    ax_right.yaxis.tick_right()
    ax_right.tick_params(labelsize=12)
    ax_right.set_xlabel('Density Residuals (m$^{-2}$)', fontsize=12)
    ax_right.legend(fontsize=12)

    # Plot in the subplot below
    ax_bottom = grid["bottom"]
    ax_bottom.plot(w_av_rho_HE, w_av_T_HE - T_s, 'r^', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(w_av_T_HE - T_s))
    ax_bottom.plot(rho_master_HE, T_master_HE - T_s, 'b.', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(T_master_HE - T_s))
    ax_bottom.plot(rho_master_HE, (T_master_HE - T_s)*0, 'k', linestyle='--')
    ax_bottom.sharex(grid["main"])
    ax_bottom.set_xlabel('Log column density (m$^{-2}$)', fontsize=12)
    ax_bottom.set_ylabel('Temp. Residuals (K)', fontsize=12)
    ax_bottom.tick_params(labelsize=12)
    ax_bottom.legend(fontsize=12)
    
    fig.delaxes(grid['bottom2'])
    # Adjust the spacing between subplots
    fig.subplots_adjust(wspace=0, hspace=0)
    if save:
        plt.savefig(save_location+'Retrieved_param_big_HE.pdf')

else:
    fig = plt.figure(figsize=[15, 10])
    # Create the grid of subplots using subplot_mosaic
    grid = fig.subplot_mosaic(
        [["main", "right"], 
         ["bottom", "bottom2"]],
        gridspec_kw={"height_ratios": [3, 1], "width_ratios": [3, 1]}
    )

    # Plot in the main plot
    ax_main = grid["main"]
    ax_main.plot(np.log10(n_s), T_s, 'ks', label='Expected', linestyle='-', markersize=12)
    ax_main.plot(w_av_rho, w_av_T, 'r^', label='Weighted Average', linestyle='-', markersize=12)
    ax_main.plot(rho_master, T_master, 'b.', label='Master', linestyle='-', markersize=12)
    ax_main.set_ylabel('Temperature (K)', fontsize=12)
    ax_main.tick_params(labelsize=12)
    ax_main.legend(fontsize=12)
    ax_main.set_title('Retrieved Parameter Grid', fontsize=12)
    
    # Plot in the subplot to the right
    ax_right = grid["right"]
    ax_right.plot(w_av_rho - np.log10(n_s), w_av_T, 'r^', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(w_av_rho - np.log10(n_s)))
    ax_right.plot(rho_master - np.log10(n_s), T_master, 'b.', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(rho_master - np.log10(n_s)))
    ax_right.plot((rho_master - np.log10(n_s))*0, T_master, 'k', linestyle='--')
    ax_right.yaxis.tick_right()
    ax_right.tick_params(labelsize=12)
    ax_right.set_xlabel('Density Residuals (m$^{-2}$)', fontsize=12)
    ax_right.legend(fontsize=12)

    # Plot in the subplot below
    ax_bottom = grid["bottom"]
    ax_bottom.plot(w_av_rho, w_av_T - T_s, 'r^', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(w_av_T - T_s))
    ax_bottom.plot(rho_master, T_master - T_s, 'b.', linestyle='-', markersize=12, label='RMS = %.3f'%np.std(T_master - T_s))
    ax_bottom.plot(rho_master, (T_master - T_s)*0, 'k', linestyle='--')
    ax_bottom.sharex(grid["main"])
    ax_bottom.set_xlabel('Log column density (m$^{-2}$)', fontsize=12)
    ax_bottom.set_ylabel('Temp. Residuals (K)', fontsize=12)
    ax_bottom.tick_params(labelsize=12)
    ax_bottom.legend(fontsize=12)
    
    fig.delaxes(grid['bottom2'])
    # Adjust the spacing between subplots
    fig.subplots_adjust(wspace=0, hspace=0)
    if save:
        plt.savefig(save_location+'Retrieved_param_big.pdf')
    